# The canonical coordinates of FWM

In this notebook we use our analytic solution to four wave mixing in the Weierstrass notation to explore coordinate transformations. We show that it is possible to transform four wave mixing to a canonical coordinate system in which the solutions are single valued functions, namely Kronecker theta functions. In doing so, we show that the classic trick of turning the quartic to a cubic in the elliptic differential equation leaves the differential system of coupled modes invariant in terms of the type of terms that appear but it delicately balances coefficients to kill the quartic term.  

In [20]:
from sympy import *

# -- Symbols --

(x, y, z, t, x0, y0, z0, z1, t0, Z, g2, g3, m, n, l, k, i, j, M, N, C, n2, T) = symbols(
    '''x, y, z, t, x0, y0, z0, z1, t0, Z, g2, g3, m, n, l, k, i, j, M, N, C, n2, T'''
)
(delta, nu, Aeff, chi) = symbols(
    '''delta, nu, Aeff, chi'''
)

gtilde2 = Symbol('gtilde2', latex_name=r'\tilde{g_2}')
gtilde3 = Symbol('gtilde3', latex_name=r'\tilde{g_3}')

# -- Functions --

pw = Function('pw') # Weierstrass P function
pwp = Function('pwp') # Derivative of Weierstrass P function
zw = Function('zw') # Weierstrass Zeta function
sigma = Function('sigma') # Weierstrass Sigma function
wp = Function('wp') # Weierstrass P function
wpp = Function('\wp\'') # Derivative of Weierstrass P function
zeta = Function('zeta') # Weierstrass Zeta function

rhop = Function('\\rho\'')
Delta = Function('Delta')
rho = Function('rho')
kappa = Function('kappa')
phi = Function('phi')
h = Function('h')
q = Function('q')
s = Function('s')
u = Function('u')
v = Function('v')
w = Function('w')
ws = Function('ws')
xi = Function('xi')
P = Function('P') # Polynomial
Q = Function('Q') # Polynomial
R = Function('R') # Polynomial

U = Function('U')
V = Function('V')
W = Function('W')
H = Function('H')


uhat = Function('uhat', latex_name=r'\hat{u}')
vhat = Function('vhat', latex_name=r'\hat{v}')
Hhat = Symbol('Hhat', latex_name=r'\hat{H}')

ubar = Function('ubar', latex_name=r'\bar{u}')
vbar = Function('vbar', latex_name=r'\bar{v}')
Hbar = Symbol('Hbar', latex_name=r'\bar{H}')

utilde = Function('utilde', latex_name=r'\tilde{u}')
vtilde = Function('vtilde', latex_name=r'\tilde{v}')
Htilde = Symbol('Htilde', latex_name=r'\tilde{H}')
htilde = Function('htilde', latex_name=r'\tilde{h}')
wtilde = Function('wtilde', latex_name=r'\tilde{w}')


wp = Function('wp')


Summ = Function('Summ')

# -- Indexed Symbols --

Omega = IndexedBase('Omega')
f = IndexedBase('f')
fc = IndexedBase('fc')
F = IndexedBase('F')
r = IndexedBase('r')
gamma = IndexedBase('gamma')
mu = IndexedBase('mu')
nu = IndexedBase('nu')
theta = IndexedBase('theta')
X = IndexedBase('X')
Y = IndexedBase('Y')
a = IndexedBase('a')
b = IndexedBase('b')
c = IndexedBase('c')
d = IndexedBase('d')
p = IndexedBase('p')
G = IndexedBase('G')
psi = IndexedBase('psi')
Psi = IndexedBase('Psi')
upsilon = IndexedBase('upsilon')
epsilon = IndexedBase('epsilon')
omega = IndexedBase('omega')
alpha = IndexedBase('alpha')
beta = IndexedBase('beta')
K = IndexedBase('K')
lam = IndexedBase('lambda')
Lam = IndexedBase('Lambda')

wild = Wild('*')


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# kth order derivatives of Weierstrass P
from wpk import wpk, wzk, wsk, run_tests

# The package containing mpmath expressions for Weierstrass elliptic functions
from weierstrass_modified import Weierstrass
we = Weierstrass()
from mpmath import exp as mpexp

# Numeric solutions to diff eqs
from numpy import linspace, absolute, angle, square, real, imag, conj, array as arraynp, concatenate
from numpy import vectorize as np_vectorize # not to get confused with vectorise in other packages
import scipy.integrate
import matplotlib.pyplot as plt
from random import random

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
uhat(z)
ubar(z)
utilde(z)

uhat(z)

ubar(z)

utilde(z)

In [22]:
def bs(eq, func):
    """Apply func to both sides"""
    return Eq(func(eq.lhs), func(eq.rhs))

## Elliptic function identities

In [23]:
sigma_p_identity = Eq(
    wp(y, g2, g3) - wp(x, g2, g3),
    sigma(x + y, g2, g3) * sigma(x - y, g2, g3) / (sigma(x, g2, g3) ** 2 * sigma(y, g2, g3) ** 2) 
)
pw_to_zw_identity = Eq(
    (wpp(z,g2,g3) - wpp(x,g2,g3))/(wp(z,g2,g3) - wp(x,g2,g3))/2,
    zeta(z + x,g2, g3) - zeta(z,g2, g3) - zeta(x,g2, g3)
)
pw_to_zw_identity_one_sided = Eq(
    wpp(z, g2, g3)/(wp(x, g2, g3) - wp(z, g2, g3)),
    2*zeta(z, g2, g3) - zeta(-x + z, g2, g3) - zeta(x + z, g2, g3)
)

zw_dlog_sigma_identity = Eq(zeta(z - x, g2, g3), Derivative(log(sigma(z - x, g2, g3)), z))
pw_to_dlog_sigma_identity = pw_to_zw_identity.subs([
    zw_dlog_sigma_identity.subs(x,-x).args,
    zw_dlog_sigma_identity.subs(x,0).args,
])
pw_to_dlog_sigma_identity_b = pw_to_dlog_sigma_identity.subs(x,-x).subs([
    (wp(-x,g2,g3), wp(x,g2,g3)), (wpp(-x,g2,g3), -wpp(x,g2,g3)), (zeta(-x, g2,g3), -zeta(x, g2,g3))
])

pw_addition_id = Eq(
    wp(x + y, g2, g3),
    -wp(x, g2, g3) - wp(y, g2, g3) + (wpp(x, g2, g3) - wpp(y, g2, g3))**2/(4*(wp(x, g2, g3) - wp(y, g2, g3))**2)
)

pwp_sigma_dbl_ratio = Eq(wpp(z,g2,g3), - sigma(2*z,g2,g3)/sigma(z,g2,g3)**4)

sigma_p_identity
pw_to_zw_identity
pw_to_zw_identity_one_sided
zw_dlog_sigma_identity
pw_to_dlog_sigma_identity_b
pw_addition_id
pwp_sigma_dbl_ratio

Eq(-wp(x, g2, g3) + wp(y, g2, g3), sigma(x - y, g2, g3)*sigma(x + y, g2, g3)/(sigma(x, g2, g3)**2*sigma(y, g2, g3)**2))

Eq((-\wp'(x, g2, g3) + \wp'(z, g2, g3))/(2*(-wp(x, g2, g3) + wp(z, g2, g3))), -zeta(x, g2, g3) - zeta(z, g2, g3) + zeta(x + z, g2, g3))

Eq(\wp'(z, g2, g3)/(wp(x, g2, g3) - wp(z, g2, g3)), 2*zeta(z, g2, g3) - zeta(-x + z, g2, g3) - zeta(x + z, g2, g3))

Eq(zeta(-x + z, g2, g3), Derivative(log(sigma(-x + z, g2, g3)), z))

Eq((\wp'(x, g2, g3) + \wp'(z, g2, g3))/(2*(-wp(x, g2, g3) + wp(z, g2, g3))), zeta(x, g2, g3) - Derivative(log(sigma(z, g2, g3)), z) + Derivative(log(sigma(-x + z, g2, g3)), z))

Eq(wp(x + y, g2, g3), (\wp'(x, g2, g3) - \wp'(y, g2, g3))**2/(4*(wp(x, g2, g3) - wp(y, g2, g3))**2) - wp(x, g2, g3) - wp(y, g2, g3))

Eq(\wp'(z, g2, g3), -sigma(2*z, g2, g3)/sigma(z, g2, g3)**4)

## The FWM system from Agrawal

*Theory of Four-Wave Mixing, p371, Nonlinear Fiber Optics 4th Edition, Govind P. Agrawal*

The book references [this paper](https://ieeexplore.ieee.org/document/1071660) but it is not open access.

## The solution in original coordinates

We start by reminding ourselves of the solutions derived in the *Four Wave mixing Case* notebook for FWM modes in the original coordinates $u,v$:

In [24]:
u_sqrt_wp_z0_z1 = Eq(
    u(z, mu[j]),
    sqrt(wpp(z1, g2, g3))*sigma(z - 2*z0 + mu[j], g2, g3)*
    exp(z*r[0, j] + log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] + epsilon[j])/
    (sqrt(wp(z1, g2, g3) - wp(z - z0, g2, g3))*sqrt(wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))
     *sigma(z - z0, g2, g3)*sigma(-z0 + mu[j], g2, g3)*d[4]**(1/4))
)
v_sqrt_wp_z0_z1 = Eq(
    v(z, mu[j]),
    sqrt(wpp(z1, g2, g3))*sigma(z - mu[j], g2, g3)*
    exp(-z*r[0, j] - log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] - epsilon[j])/
    (sqrt(wp(z1, g2, g3) - wp(z - z0, g2, g3))*sqrt(wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))
     *sigma(z - z0, g2, g3)*sigma(-z0 + mu[j], g2, g3)*d[4]**(1/4))
)

u_sqrt_wp_z0_z1
v_sqrt_wp_z0_z1

Eq(u(z, mu[j]), sqrt(\wp'(z1, g2, g3))*sigma(z - 2*z0 + mu[j], g2, g3)*exp(z*r[0, j] + log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] + epsilon[j])/(sqrt(wp(z1, g2, g3) - wp(z - z0, g2, g3))*sqrt(wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))*sigma(z - z0, g2, g3)*sigma(-z0 + mu[j], g2, g3)*d[4]**0.25))

Eq(v(z, mu[j]), sqrt(\wp'(z1, g2, g3))*sigma(z - mu[j], g2, g3)*exp(-z*r[0, j] - log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] - epsilon[j])/(sqrt(wp(z1, g2, g3) - wp(z - z0, g2, g3))*sqrt(wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))*sigma(z - z0, g2, g3)*sigma(-z0 + mu[j], g2, g3)*d[4]**0.25))

### The original parameters

In [25]:
b_j_coeffs = [
    Eq(b[0], a[0] + Sum(a[j]*gamma[j], (j,1,4)) + Sum(a[j,k]/2*gamma[j]*gamma[k], (j,1,4), (k,1,4))),
    Eq(b[1], -Sum(a[j,k]/2*(gamma[j] + gamma[k]), (j,1,4), (k,1,4)) - Sum(a[j], (j,1,4))),
    Eq(b[2], Sum(a[j,k]/2, (j,1,4), (k,1,4)))
]

# Depend on gamma[j]
sum_gamma_j_0 = Eq(Sum(gamma[j], (j,1,4)), 0)
c_j_coeffs = [
    Eq(c[0], Product(gamma[j], (j,1,4))),
    Eq(c[1], -Sum(Product(gamma[j], (j, 1, k-1))*Product(gamma[j], (j, k+1, 4)), (k, 1, 4))),
    Eq(c[2], -Sum(((gamma[j] - gamma[k])**2)/4,(j,1,4), (k,1,4)) + 3*Sum((gamma[j]**2)/2, (j,1,4))),
    Eq(c[3], -Sum(gamma[j], (j,1,4))).subs([sum_gamma_j_0.args]),
    Eq(c[4], 1)
]

d_j_coeffs = [
    Eq(d[0], b[0]**2 - 4*c[0]),
    Eq(d[1], 2*b[0]*b[1] - 4*c[1]),
    Eq(d[2], 2*b[0]*b[2] + b[1]**2 - 4*c[2]),
    Eq(d[3], 2*b[1]*b[2] - 4*c[3]),
    Eq(d[4], b[2]**2 - 4*c[4])
]


g2_dj = Eq(g2, d[0]*d[4] - d[1]*d[3]/4 + d[2]**2/12)
g3_dj = Eq(g3, d[0]*d[2]*d[4]/6 - d[0]*d[3]**2/16 - d[1]**2*d[4]/16 + d[1]*d[2]*d[3]/48 - d[2]**3/216)

for _eq in b_j_coeffs:
    _eq
    
for _eq in c_j_coeffs:
    _eq
    
for _eq in d_j_coeffs:
    _eq
    
g2_dj
g3_dj

Eq(b[0], a[0] + Sum(a[j]*gamma[j], (j, 1, 4)) + Sum(a[j, k]*gamma[j]*gamma[k]/2, (j, 1, 4), (k, 1, 4)))

Eq(b[1], -Sum(a[j], (j, 1, 4)) - Sum((gamma[j] + gamma[k])*a[j, k]/2, (j, 1, 4), (k, 1, 4)))

Eq(b[2], Sum(a[j, k]/2, (j, 1, 4), (k, 1, 4)))

Eq(c[0], Product(gamma[j], (j, 1, 4)))

Eq(c[1], -Sum(Product(gamma[j], (j, 1, k - 1))*Product(gamma[j], (j, k + 1, 4)), (k, 1, 4)))

...

Eq(d[2], 2*b[0]*b[2] + b[1]**2 - 4*c[2])

Eq(d[3], 2*b[1]*b[2] - 4*c[3])

Eq(d[4], b[2]**2 - 4*c[4])

Eq(g2, d[0]*d[4] - d[1]*d[3]/4 + d[2]**2/12)

Eq(g3, d[0]*d[2]*d[4]/6 - d[0]*d[3]**2/16 - d[1]**2*d[4]/16 + d[1]*d[2]*d[3]/48 - d[2]**3/216)

We recall the equations from the original derivation of the solution in terms of $\rho, b_j, \gamma_j, \lambda_1$:

In [26]:
drhop_b = Eq(
    Derivative(rho(z), z)**2,
    (rho(z)**2*b[2] + rho(z)*b[1] + b[0])**2 - 4*Product(-rho(z) + gamma[j], (j, 1, 4))
)
drhop_d = Eq(Derivative(rho(z), z)**2, Sum(rho(z)**j*d[j], (j, 0, 4)))

drho_2z = Eq(Derivative(rho(z), (z, 2)), Sum(j*rho(z)**(j-1)*d[j]/2, (j, 0, 4)))
drho_2z_b = Eq(
    Derivative(rho(z), (z, 2)),
    (2*rho(z)*b[2] + b[1])*(rho(z)**2*b[2] + rho(z)*b[1] + b[0]) 
    + 2*Product(-rho(z) + gamma[j], (j, 1, 4))*Sum(
        1/(-rho(z) + gamma[k]), 
        (k, 1, 4)
    )
)

drhop_b
drhop_d
drho_2z
drho_2z_b

Eq(Derivative(rho(z), z)**2, (rho(z)**2*b[2] + rho(z)*b[1] + b[0])**2 - 4*Product(-rho(z) + gamma[j], (j, 1, 4)))

Eq(Derivative(rho(z), z)**2, Sum(rho(z)**j*d[j], (j, 0, 4)))

Eq(Derivative(rho(z), (z, 2)), Sum(j*rho(z)**(j - 1)*d[j]/2, (j, 0, 4)))

Eq(Derivative(rho(z), (z, 2)), (2*rho(z)*b[2] + b[1])*(rho(z)**2*b[2] + rho(z)*b[1] + b[0]) + 2*Product(-rho(z) + gamma[j], (j, 1, 4))*Sum(1/(-rho(z) + gamma[k]), (k, 1, 4)))

We define three parameters for convenience namely $d_5, C, \chi$:

In [27]:
# C, b, gamma, lam1 relation

lam1_root = Eq(0, drhop_d.rhs.subs(rho(z), lam[1]).doit())
gamma_j_prod_b_poly = Eq(drhop_b.rhs.subs(rho(z), lam[1]),0)
gamma_j_prod_b_poly = Eq(
    (gamma_j_prod_b_poly.lhs.args[0] - gamma_j_prod_b_poly.lhs)/4,
    (gamma_j_prod_b_poly.lhs.args[0] - gamma_j_prod_b_poly.rhs)/4
)

C_gam_prod = Eq(Product(sqrt(lam[1]-gamma[l]),(l,1,4)), C)
C_gam_prod_sqrd = Eq(
    C_gam_prod.rhs.doit()**2, 
    (C_gam_prod.lhs.doit()**2).expand()
    -gamma_j_prod_b_poly.lhs.doit().expand() + gamma_j_prod_b_poly.rhs
)
C_gam_prod_sign = Eq(b[0] + b[1]*lam[1] + b[2]*lam[1]**2, (-1)**m*2*C)

# d5 relations
d5_lam1 = Eq(d[5], d[1] + 2*d[2]*lam[1] + 3*d[3]*lam[1]**2 + 4*d[4]*lam[1]**3)
d5_gam_lam1 = drho_2z_b.subs([drho_2z.args, (rho(z),lam[1])])
d5_gam_lam1 = Eq(
    d5_gam_lam1.lhs.doit().subs(d5_lam1.rhs/2, d5_lam1.lhs/2),
    d5_gam_lam1.rhs.subs([gamma_j_prod_b_poly.args])
)
d5_gam_lam1 = Eq(d5_gam_lam1.lhs*2, d5_gam_lam1.rhs*2)
d5_gam_lam1_C = d5_gam_lam1.subs([C_gam_prod_sign.args, ((-1)**(2*m),1)])

# chi 

chi_d5_C = Eq(chi, C_gam_prod_sign.rhs*8/d[5])
chi_d5_b = Eq(chi, C_gam_prod_sign.lhs*8/d[5])
C_d5_chi = Eq(C, solve(chi_d5_C, C)[0])


lam1_root
gamma_j_prod_b_poly
C_gam_prod
C_gam_prod_sqrd
C_gam_prod_sign

d5_lam1
d5_gam_lam1
d5_gam_lam1_C

chi_d5_C
chi_d5_b
C_d5_chi

Eq(0, d[0] + d[1]*lambda[1] + d[2]*lambda[1]**2 + d[3]*lambda[1]**3 + d[4]*lambda[1]**4)

Eq(Product(gamma[j] - lambda[1], (j, 1, 4)), (b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)**2/4)

Eq(Product(sqrt(-gamma[l] + lambda[1]), (l, 1, 4)), C)

Eq(C**2, (b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)**2/4)

Eq(b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2, 2*(-1)**m*C)

...

Eq(d[5], 2*(b[1] + 2*b[2]*lambda[1])*(b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2) + (b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)**2*Sum(1/(gamma[k] - lambda[1]), (k, 1, 4)))

Eq(d[5], 4*(-1)**m*C*(b[1] + 2*b[2]*lambda[1]) + 4*C**2*Sum(1/(gamma[k] - lambda[1]), (k, 1, 4)))

Eq(chi, 16*(-1)**m*C/d[5])

Eq(chi, (8*b[0] + 8*b[1]*lambda[1] + 8*b[2]*lambda[1]**2)/d[5])

Eq(C, chi*d[5]/(16*(-1)**m))

In [28]:
gam_sum_d5_1 = Eq(
    Sum(1/(gamma[k] - lam[1]), (k, 1, 4)),
    (
        (d[5] - 2*(b[1] + 2*b[2]*lam[1])*(b[0] + b[1]*lam[1] + b[2]*lam[1]**2))/
        ((b[0] + b[1]*lam[1] + b[2]*lam[1]**2)**2)
    ).subs(*d5_lam1.args)
)
gam_sum_d5_2 = Eq(diff(gam_sum_d5_1.lhs, lam[1]), diff(gam_sum_d5_1.rhs, lam[1]))

gam_sum_d5_1_C = gam_sum_d5_1.subs([C_gam_prod_sign.args, (d5_lam1.rhs, d5_lam1.lhs)]) .subs([
        ((-1)**(-2*m),1), ((-1)**(2*m),1), ((-1)**(-3*m),(-1)**m)
    ])
gam_sum_d5_2_C = gam_sum_d5_2.subs([C_gam_prod_sign.args, (d5_lam1.rhs, d5_lam1.lhs)])
gam_sum_d5_2_C = Eq(
    gam_sum_d5_2_C.lhs,
    gam_sum_d5_2_C.rhs
    .expand().collect(lam[1], simplify)
    .subs([
        ((-1)**(-2*m),1), ((-1)**(2*m),1), ((-1)**(-3*m),(-1)**m)
    ])
    .subs([_.args for _ in d_j_coeffs])
    .subs([c_j_coeffs[3].args, c_j_coeffs[4].args])
    .expand().collect(C, factor)
    .subs([
        ((-1)**(-2*m),1), ((-1)**(2*m),1), ((-1)**(-3*m),(-1)**m),
        (
            ((5*C_gam_prod_sign.lhs - 5*b[0])*b[2]).expand(), 
            ((5*C_gam_prod_sign.rhs - 5*b[0])*b[2]).expand()
        )
    ])
    .expand().collect(C, factor)
)

gam_sum_d5_1
gam_sum_d5_2

gam_sum_d5_1_C
gam_sum_d5_2_C

Eq(Sum(1/(gamma[k] - lambda[1]), (k, 1, 4)), (-(2*b[1] + 4*b[2]*lambda[1])*(b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2) + d[1] + 2*d[2]*lambda[1] + 3*d[3]*lambda[1]**2 + 4*d[4]*lambda[1]**3)/(b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)**2)

Eq(Sum((gamma[k] - lambda[1])**(-2), (k, 1, 4)), (-2*b[1] - 4*b[2]*lambda[1])*(-(2*b[1] + 4*b[2]*lambda[1])*(b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2) + d[1] + 2*d[2]*lambda[1] + 3*d[3]*lambda[1]**2 + 4*d[4]*lambda[1]**3)/(b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)**3 + ((-2*b[1] - 4*b[2]*lambda[1])*(b[1] + 2*b[2]*lambda[1]) - 4*(b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)*b[2] + 2*d[2] + 6*d[3]*lambda[1] + 12*d[4]*lambda[1]**2)/(b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)**2)

Eq(Sum(1/(gamma[k] - lambda[1]), (k, 1, 4)), (-2*(-1)**m*C*(2*b[1] + 4*b[2]*lambda[1]) + d[5])/(4*C**2))

Eq(Sum((gamma[k] - lambda[1])**(-2), (k, 1, 4)), 8*(-1)**m*b[2]/C - (-1)**m*(b[1] + 2*b[2]*lambda[1])*d[5]/(4*C**3) + (-4*b[0]*b[2] + b[1]**2 - 2*c[2] - 12*lambda[1]**2)/C**2)

In [29]:
# Utility function - both C, d[5] are polynomials in lam[1]

lam1_C_d5_bc_eq = Eq(
    (
        ((C_gam_prod_sign.lhs - C_gam_prod_sign.rhs)*lam[1]/b[2]).expand() -
        ((d5_lam1.rhs - d5_lam1.lhs)/d[4]/4).expand()
    ).subs([(lam[1]**2, solve(C_gam_prod_sign, lam[1]**2)[0])])
    .expand().collect(lam[1], factor)
    ,
    0
)

lam1_C_d5_bc = Eq(lam[1], solve(lam1_C_d5_bc_eq, lam[1])[0])

lam1_C_d5_bc

Eq(lambda[1], (8*(-1)**m*C*b[1]*d[4] - 6*(-1)**m*C*b[2]*d[3] - 4*b[0]*b[1]*d[4] + 3*b[0]*b[2]*d[3] - b[2]**2*d[1] + b[2]**2*d[5])/(8*(-1)**m*C*b[2]*d[4] - 4*b[0]*b[2]*d[4] + 4*b[1]**2*d[4] - 3*b[1]*b[2]*d[3] + 2*b[2]**2*d[2]))

### Reorganising the original equations of motion

In [30]:
a_0_eq = Eq(
    a[0]
    + Sum(a[j]*u(z,mu[j])*v(z,mu[j]), (j,1,4))
    + Sum(a[j,k]/2*u(z,mu[j])*v(z,mu[j])*u(z,mu[k])*v(z,mu[k]), (j,1,4), (k,1,4))
    ,
    Product(u(z,mu[j]),(j,1,4)) + Product(v(z,mu[j]),(j,1,4))
).replace(j,l)

du_phase_mod_part = (a[j] + Sum(a[j,k]*u(z,mu[k])*v(z,mu[k]), (k,1,4)))*u(z,mu[j])
du_mixing_part = Product(v(z,mu[k]), (k,1,4))/v(z, mu[j])
duj = Eq(diff(u(z,mu[j]),z), -du_phase_mod_part + du_mixing_part)

dv_phase_mod_part = -(a[j] + Sum(a[j,k]*u(z,mu[k])*v(z,mu[k]), (k,1,4)))*v(z,mu[j])
dv_mixing_part = -Product(u(z,mu[k]), (k,1,4))/u(z, mu[j])
dvj = Eq(diff(v(z,mu[j]),z), (-dv_phase_mod_part).factor() + dv_mixing_part)

Quv_eq = Eq(
    Q(u(z, mu[1])*v(z, mu[1]), u(z, mu[2])*v(z, mu[2]), u(z, mu[3])*v(z, mu[3]), u(z, mu[4])*v(z, mu[4])),
    a[0] + Sum(a[j]*u(z, mu[j])*v(z, mu[j]), (j,1,4)) + 
    Sum(a[j,k]*u(z, mu[j])*v(z, mu[j])*u(z, mu[k])*v(z, mu[k]), (j,1,4), (j,1,4))
)

uv_j_rho = Eq(u(z,mu[j])*v(z,mu[j]), gamma[j] - rho(z))

duvj = Eq(
    Derivative(u(z, mu[j])*v(z, mu[j]),z), 
    Product(v(z, mu[k]), (k, 1, 4)) - Product(u(z, mu[k]), (k, 1, 4))
)

uprodj = Eq((a_0_eq.rhs - duvj.rhs.replace(k,l))/2, sum(factor(_/2) for _ in (a_0_eq.lhs - duvj.lhs).args))
vprodj = Eq((a_0_eq.rhs + duvj.rhs.replace(k,l))/2, sum(factor(_/2) for _ in (a_0_eq.lhs + duvj.lhs).args))

dlog_u_no_wm = Eq(
    Derivative(u(z, mu[j]), z)/u(z, mu[j]),
    -a[j] - Sum(u(z, mu[k])*v(z, mu[k])*a[j, k], (k, 1, 4)) 
    + Derivative(u(z, mu[j])*v(z, mu[j]), z)/(2*u(z, mu[j])*v(z, mu[j])) 
    + a[0]/(2*u(z, mu[j])*v(z, mu[j])) 
    + Sum(
        u(z, mu[k])*v(z, mu[k])*u(z, mu[l])*v(z, mu[l])*a[l, k], 
        (l, 1, 4), 
        (k, 1, 4)
    )/(4*u(z, mu[j])*v(z, mu[j])) 
    + Sum(
        u(z, mu[l])*v(z, mu[l])*a[l], 
        (l, 1, 4)
    )/(2*u(z, mu[j])*v(z, mu[j]))
)

dlog_u_no_wm_check = dlog_u_no_wm.doit().subs([
    (a[0], solve(a_0_eq,a[0])[0]),
    duj.args,
    dvj.args
]).simplify()
if not dlog_u_no_wm_check.doit().simplify():
    raise ValueError('dlog_u_no_wm_check failed')

ajk_syms = [(a[_j,_k], a[_k,_j]) for _j in range(1,5) for _k in range(1,5) if _j > _k]

    
uv_j_rho
duvj
a_0_eq
uprodj
vprodj
dlog_u_no_wm

Eq(u(z, mu[j])*v(z, mu[j]), -rho(z) + gamma[j])

Eq(Derivative(u(z, mu[j])*v(z, mu[j]), z), -Product(u(z, mu[k]), (k, 1, 4)) + Product(v(z, mu[k]), (k, 1, 4)))

Eq(a[0] + Sum(u(z, mu[l])*v(z, mu[l])*a[l], (l, 1, 4)) + Sum(u(z, mu[k])*u(z, mu[l])*v(z, mu[k])*v(z, mu[l])*a[l, k]/2, (l, 1, 4), (k, 1, 4)), Product(u(z, mu[l]), (l, 1, 4)) + Product(v(z, mu[l]), (l, 1, 4)))

Eq(Product(u(z, mu[l]), (l, 1, 4)), -Derivative(u(z, mu[j])*v(z, mu[j]), z)/2 + a[0]/2 + Sum(u(z, mu[k])*v(z, mu[k])*Sum(u(z, mu[l])*v(z, mu[l])*a[l, k], (l, 1, 4)), (k, 1, 4))/4 + Sum(u(z, mu[l])*v(z, mu[l])*a[l], (l, 1, 4))/2)

Eq(Product(v(z, mu[l]), (l, 1, 4)), Derivative(u(z, mu[j])*v(z, mu[j]), z)/2 + a[0]/2 + Sum(u(z, mu[k])*v(z, mu[k])*Sum(u(z, mu[l])*v(z, mu[l])*a[l, k], (l, 1, 4)), (k, 1, 4))/4 + Sum(u(z, mu[l])*v(z, mu[l])*a[l], (l, 1, 4))/2)

Eq(Derivative(u(z, mu[j]), z)/u(z, mu[j]), -a[j] - Sum(u(z, mu[k])*v(z, mu[k])*a[j, k], (k, 1, 4)) + Derivative(u(z, mu[j])*v(z, mu[j]), z)/(2*u(z, mu[j])*v(z, mu[j])) + a[0]/(2*u(z, mu[j])*v(z, mu[j])) + Sum(u(z, mu[l])*v(z, mu[l])*a[l], (l, 1, 4))/(2*u(z, mu[j])*v(z, mu[j])) + Sum(u(z, mu[k])*u(z, mu[l])*v(z, mu[k])*v(z, mu[l])*a[l, k], (l, 1, 4), (k, 1, 4))/(4*u(z, mu[j])*v(z, mu[j])))

## Gauge transform to the hat coordinates

In this section we implement a gauge transform to remove XPM, i.e., to remove the log of $\sigma$ terms from the exponential:

In [31]:
gauge_phi = Eq(
    phi(z, j), 
    (a[j] - gamma[j]*Sum(a[l, k]/2, (l, 1, 4), (k, 1, 4))/2 - Sum(a[l], (l, 1, 4))/4)*z +
    Sum((a[j, k] - Sum(a[l, k]/4, (l, 1, 4)))*Integral(uhat(z, mu[k])*vhat(z, mu[k]),z), (k, 1, 4))
)
uU_sub = Eq(
    u(z,mu[j]), 
    uhat(z, mu[j])
    *exp(-phi(z,j))
)
vV_sub = Eq(
    v(z,mu[j]), 
    vhat(z, mu[j])
    *exp(phi(z,j))
)
uuVV_sub = Eq(uU_sub.lhs*vV_sub.lhs, (uU_sub.rhs*vV_sub.rhs).simplify())
gauge_phi_sum = Eq(
    Sum(phi(z,j),(j,1,4)), 
    Sum(phi(z,j).subs([gauge_phi.args]),(j,1,4)).doit()
    .subs(gamma[4], - gamma[1] - gamma[2] - gamma[3]).simplify()
)
d_gauge_phi = Eq(diff(gauge_phi.lhs,z), diff(gauge_phi.rhs,z))

uU_sub
vV_sub
uuVV_sub
gauge_phi
d_gauge_phi
gauge_phi_sum

Eq(u(z, mu[j]), uhat(z, mu[j])*exp(-phi(z, j)))

Eq(v(z, mu[j]), vhat(z, mu[j])*exp(phi(z, j)))

Eq(u(z, mu[j])*v(z, mu[j]), uhat(z, mu[j])*vhat(z, mu[j]))

Eq(phi(z, j), z*(a[j] - gamma[j]*Sum(a[l, k]/2, (l, 1, 4), (k, 1, 4))/2 - Sum(a[l], (l, 1, 4))/4) + Sum((a[j, k] - Sum(a[l, k]/4, (l, 1, 4)))*Integral(uhat(z, mu[k])*vhat(z, mu[k]), z), (k, 1, 4)))

Eq(Derivative(phi(z, j), z), a[j] - gamma[j]*Sum(a[l, k]/2, (l, 1, 4), (k, 1, 4))/2 + Sum((a[j, k] - Sum(a[l, k]/4, (l, 1, 4)))*uhat(z, mu[k])*vhat(z, mu[k]), (k, 1, 4)) - Sum(a[l], (l, 1, 4))/4)

Eq(Sum(phi(z, j), (j, 1, 4)), 0)

Intermodal power conservation is invariant:

In [32]:
uv_gamma_cons =Eq(
    uv_j_rho.lhs - uv_j_rho.lhs.subs(j,k),
    uv_j_rho.rhs - uv_j_rho.rhs.subs(j,k)
)

uv_gamma_cons_gauge = Eq(
    uv_gamma_cons.lhs.subs([
        uU_sub.args, 
        vV_sub.args,
        uU_sub.subs(j,k).args, 
        vV_sub.subs(j,k).args
    ]).simplify(),
    uv_gamma_cons.rhs
)
uv_gam_gauge_k_to_j = Eq(
    uhat(z, mu[j])*vhat(z, mu[j]) - uv_gamma_cons_gauge.lhs,
    uhat(z, mu[j])*vhat(z, mu[j]) - uv_gamma_cons_gauge.rhs
)


uv_j_rho
uv_gamma_cons
uv_gamma_cons_gauge

Eq(u(z, mu[j])*v(z, mu[j]), -rho(z) + gamma[j])

Eq(u(z, mu[j])*v(z, mu[j]) - u(z, mu[k])*v(z, mu[k]), gamma[j] - gamma[k])

Eq(uhat(z, mu[j])*vhat(z, mu[j]) - uhat(z, mu[k])*vhat(z, mu[k]), gamma[j] - gamma[k])

In [33]:
xpm_to_spm = Eq(
    Sum(uhat(z, mu[k])*vhat(z, mu[k])*Sum(a[l, k]/4, (l, 1, 4)), (k, 1, 4)),
    uhat(z, mu[j])*vhat(z, mu[j])*Sum(a[l, k]/4, (l, 1, 4), (k, 1, 4)) -
    gamma[j]*Sum(a[l, k]/2, (l, 1, 4), (k, 1, 4))/2 +
    Sum(gamma[k]*Sum(a[l, k]/4, (l, 1, 4)), (k, 1, 4))
)

ajk_to_b1_sum = Eq( 
    Sum(gamma[k]*Sum(a[l, k]/4, (l, 1, 4)), (k, 1, 4)) + Sum(a[l], (l, 1, 4))/4,
    (
        Sum(gamma[k]*Sum(a[l, k]/4, (l, 1, 4)), (k, 1, 4)) + Sum(a[l], (l, 1, 4))/4
        + b_j_coeffs[1].rhs/4 - b_j_coeffs[1].lhs/4
    ).doit().simplify().subs(ajk_syms)

)

xpm_to_spm_check = (xpm_to_spm.lhs - xpm_to_spm.rhs).doit().subs([
    (uhat(z, mu[_k+1])*vhat(z, mu[_k+1]), gamma[_k+1] - rho(z)) for _k in range(4)
]).subs([(uhat(z, mu[j])*vhat(z, mu[j]), gamma[j] - rho(z))]).expand() == 0
if not xpm_to_spm_check:
    raise

gauge_subs = [
    gauge_phi_sum.doit().args, 
    (diff(gauge_phi.lhs, z), diff(gauge_phi.rhs, z)),
    (
        Sum(uhat(z, mu[k])*vhat(z, mu[k])*a[j, k], (k,1,4)).doit(), 
        Sum(uhat(z, mu[k])*vhat(z, mu[k])*a[j, k], (k,1,4))
    ),
    (
        Sum((a[j, k] - Sum(a[l, k]/4, (l, 1, 4)))*uhat(z, mu[k])*vhat(z, mu[k]), (k, 1, 4))
        - Sum(uhat(z, mu[k])*vhat(z, mu[k])*a[j, k], (k, 1, 4)),
        -Sum(Sum(a[l, k]/4, (l, 1, 4))*uhat(z, mu[k])*vhat(z, mu[k]), (k, 1, 4))
    ),
    xpm_to_spm.args,
    (
        Sum(a[l,k]/4, (l,1,4), (k,1,4)),
        (Sum(a[l,k]/4, (l,1,4), (k,1,4)) - b_j_coeffs[2].rhs/2 + b_j_coeffs[2].lhs/2).doit()
    ),
    ajk_to_b1_sum.args
]

dUj = Eq(
    diff(uhat(z,mu[j]),z), 
    solve(
        duj
        .replace(*uU_sub.args)
        .replace(*vV_sub.args)
        .replace(*uU_sub.subs(j,k).args)
        .replace(*vV_sub.subs(j,k).args)
        .replace(*uU_sub.subs(j,l).args)
        .replace(*vV_sub.subs(j,l).args)
        .doit(), 
        diff(uhat(z,mu[j]),z)
    )[0].simplify()
    .expand()
    .collect(uhat(z,mu[j]), simplify)
    .subs(gauge_subs)
)
dVj = Eq(
    diff(vhat(z,mu[j]),z), 
    solve(
        dvj
        .replace(*uU_sub.args)
        .replace(*vV_sub.args)
        .replace(*uU_sub.subs(j,k).args)
        .replace(*vV_sub.subs(j,k).args)
        .replace(*uU_sub.subs(j,l).args)
        .replace(*vV_sub.subs(j,l).args)
        .doit(), 
        diff(vhat(z,mu[j]),z)
    )[0].simplify()
    .expand()
    .collect(vhat(z,mu[j]), simplify)
    .subs(gauge_subs)
)

dUVj = Eq(
    Derivative(uhat(z, mu[j])*vhat(z, mu[j]),z), 
    Derivative(uhat(z, mu[j])*vhat(z, mu[j]),z).doit().subs([dUj.args, dVj.args]).simplify()
)

dUj
dVj
dUVj

Eq(Derivative(uhat(z, mu[j]), z), (-uhat(z, mu[j])*vhat(z, mu[j])*b[2]/2 + b[1]/4)*uhat(z, mu[j]) + vhat(z, mu[1])*vhat(z, mu[2])*vhat(z, mu[3])*vhat(z, mu[4])/vhat(z, mu[j]))

Eq(Derivative(vhat(z, mu[j]), z), (uhat(z, mu[j])*vhat(z, mu[j])*b[2]/2 - b[1]/4)*vhat(z, mu[j]) - uhat(z, mu[1])*uhat(z, mu[2])*uhat(z, mu[3])*uhat(z, mu[4])/uhat(z, mu[j]))

Eq(Derivative(uhat(z, mu[j])*vhat(z, mu[j]), z), -uhat(z, mu[1])*uhat(z, mu[2])*uhat(z, mu[3])*uhat(z, mu[4]) + vhat(z, mu[1])*vhat(z, mu[2])*vhat(z, mu[3])*vhat(z, mu[4]))

In comparrison to the original system, we see that we have removed all XPM and now have a single SPM parameter $b_2$ for all modes and a single phase velocity parameter $b_1$.

In [34]:
duj
dvj

Eq(Derivative(u(z, mu[j]), z), -(a[j] + Sum(u(z, mu[k])*v(z, mu[k])*a[j, k], (k, 1, 4)))*u(z, mu[j]) + Product(v(z, mu[k]), (k, 1, 4))/v(z, mu[j]))

Eq(Derivative(v(z, mu[j]), z), (a[j] + Sum(u(z, mu[k])*v(z, mu[k])*a[j, k], (k, 1, 4)))*v(z, mu[j]) - Product(u(z, mu[k]), (k, 1, 4))/u(z, mu[j]))

We note that the value of the Hamiltonian is invariant under the gauge transform but this function is no longer the canonical Hamiltonian of the transformed equations of motion:

In [35]:
uU_prod = Eq(
    Product(u(z, mu[l]),(l,1,4)), 
    Product(uhat(z, mu[l]),(l,1,4))*
    (
        Product(u(z, mu[l]).subs([uU_sub.subs(j,l).args]),(l,1,4))/
        Product(uhat(z, mu[l]),(l,1,4))
    ).doit().simplify().subs([gauge_phi_sum.doit().args])
    
)
vV_prod = Eq(
    Product(v(z, mu[l]),(l,1,4)), 
    Product(vhat(z, mu[l]),(l,1,4))*
    (
        Product(v(z, mu[l]).subs([vV_sub.subs(j,l).args]),(l,1,4))/
        Product(vhat(z, mu[l]),(l,1,4))
    ).doit().simplify().subs([gauge_phi_sum.doit().args])
    
)

a_0_eq_UV = (
    a_0_eq
    .replace(uuVV_sub.subs(j,l).lhs*a[l], uuVV_sub.subs(j,l).rhs*a[l])
    .replace(
        uuVV_sub.subs(j,l).lhs*uuVV_sub.subs(j,k).lhs*a[l,k]/2, 
        uuVV_sub.subs(j,l).rhs*uuVV_sub.subs(j,k).rhs*a[l,k]/2
    )
    .subs([uU_prod.args, vV_prod.args])
)

dUdVj_subs = [dUj.subs(j,_j+1).args for _j in range(4)]
dUdVj_subs += [dVj.subs(j,_j+1).args for _j in range(4)]
uv_hats_to_rhos = [
     (uhat(z, mu[_j+1])*vhat(z, mu[_j+1]), gamma[_j+1] - rho(z)) for _j in range(4)
 ]
no_gam_4 = Eq(gamma[4], - gamma[1] - gamma[2] - gamma[3])
a_0_eq_UV_conserved_check_val = (
    diff(a_0_eq_UV.rhs - a_0_eq_UV.lhs + a[0],z)
    .doit().subs(dUdVj_subs)
    .doit().expand().simplify()
    .collect([
        a[_j+1,_k+1] for _j in range(4) for _k in range(4)
    ], factor)
    .subs(ajk_syms)
    .subs([b_j_coeffs[1].args, b_j_coeffs[2].args]).doit()
     .subs(ajk_syms)
     .subs(uv_hats_to_rhos)
     .subs(*no_gam_4.args)
     .simplify()
    )
a_0_eq_UV_conserved_check = a_0_eq_UV_conserved_check_val == 0

if not a_0_eq_UV_conserved_check:
    raise ValueError('a_0_eq_UV_conserved_check failed')

a_0_eq_UV

Eq(a[0] + Sum(uhat(z, mu[l])*vhat(z, mu[l])*a[l], (l, 1, 4)) + Sum(uhat(z, mu[k])*uhat(z, mu[l])*vhat(z, mu[k])*vhat(z, mu[l])*a[l, k]/2, (l, 1, 4), (k, 1, 4)), Product(uhat(z, mu[l]), (l, 1, 4)) + Product(vhat(z, mu[l]), (l, 1, 4)))

However, we can easily construct a new canonical Hamiltonian in the new coordinates. We denote the new Hamiltonian as $\hat{H}$ and it is a conserved quantity.

In [36]:
H_uv_hat = Eq(
    Hhat, 
    - Sum((b[2]*(uhat(z, mu[l])*vhat(z, mu[l]))**2 - b[1]*uhat(z, mu[l])*vhat(z, mu[l]))/4, (l, 1, 4))
    + Product(uhat(z, mu[l]), (l, 1, 4)) + Product(vhat(z, mu[l]), (l, 1, 4))
)
dUdVj_hat_subs = [dUj.subs(j,_j+1).args for _j in range(4)] + [dVj.subs(j,_j+1).args for _j in range(4)]
H_uv_hat_check = (
    diff(H_uv_hat.rhs.doit(),z).doit()
    .expand()
    .subs(dUdVj_hat_subs).simplify()
    .subs(ajk_syms).simplify()
)
if H_uv_hat_check != 0:
    raise ValueError('H_uv_hat_check not conserved')
    
    
for _j in range(4):
    if (
        diff(H_uv_hat.rhs.doit(), vhat(z, mu[_j+1])) 
        - dUj.rhs.subs(j, _j+1)
    ).doit().subs(ajk_syms).simplify() !=0:
        raise ValueError('H_uv_hat EOMs different in uhat')
        
for _j in range(4):
    if (
        - diff(H_uv_hat.rhs.doit(), uhat(z, mu[_j+1])) 
        - dVj.rhs.subs(j, _j+1)
    ).doit().subs(ajk_syms).simplify() !=0:
        raise ValueError('H_uv_hat EOMs different in vhat')

H_uv_hat


Eq(Hhat, Product(uhat(z, mu[l]), (l, 1, 4)) + Product(vhat(z, mu[l]), (l, 1, 4)) - Sum(uhat(z, mu[l])**2*vhat(z, mu[l])**2*b[2]/4 - uhat(z, mu[l])*vhat(z, mu[l])*b[1]/4, (l, 1, 4)))

In [37]:
H_uv_hat_orig = Eq(
    H_uv_hat.lhs, 
    (H_uv_hat.rhs - (a_0_eq_UV.rhs - a_0_eq_UV.lhs))
)

H_uv_hat_orig_gam = (
    H_uv_hat_orig
    .replace(uhat(z, mu[l])*vhat(z, mu[l])*a[l], gamma[l]*a[l])
    .replace(-uhat(z, mu[l])*vhat(z, mu[l])*b[1]/4, gamma[l]*b[1]/4)
    .replace((uhat(z, mu[l])*vhat(z, mu[l]))**2*b[2]/4, gamma[l]**2*b[2]/4)
    .replace(
        uhat(z, mu[k])*uhat(z, mu[l])*vhat(z, mu[k])*vhat(z, mu[l])*a[l, k]/2,
        gamma[l]*gamma[k]*a[l, k]/2
    )
    .subs([
        (
            Sum(b[1]*gamma[l]/4 + b[2]*gamma[l]**2/4, (l, 1, 4)),
            b[2]*Sum(gamma[l]**2/4, (l, 1, 4))
        )
    ])
)

H_uv_hat_orig_gam = Eq(
    H_uv_hat_orig_gam.lhs,
    H_uv_hat_orig_gam.rhs - b_j_coeffs[0].rhs.replace(j,l) + b_j_coeffs[0].lhs.replace(j,l)
)

H_uv_hat_orig
H_uv_hat_orig_gam

Eq(Hhat, a[0] + Sum(uhat(z, mu[l])*vhat(z, mu[l])*a[l], (l, 1, 4)) - Sum(uhat(z, mu[l])**2*vhat(z, mu[l])**2*b[2]/4 - uhat(z, mu[l])*vhat(z, mu[l])*b[1]/4, (l, 1, 4)) + Sum(uhat(z, mu[k])*uhat(z, mu[l])*vhat(z, mu[k])*vhat(z, mu[l])*a[l, k]/2, (l, 1, 4), (k, 1, 4)))

Eq(Hhat, b[0] - b[2]*Sum(gamma[l]**2/4, (l, 1, 4)))

## Normalised modes as Kronecker theta functions

We will now implement the transformation to the normalised modes to yield the canonical cubic form. 

### Normalised coordinates $U, V$
We start with the definition of the current modes $\hat{u}, \hat{v}$ in terms of the new modes $U,V$, and the definition of the normalising function $h(z)$ in terms of $U,V$:

In [44]:
UV_to_uv_lam1_hs = [
    Eq(
        uhat(z, mu[j]),
        2*I*sqrt(-gamma[j] + lam[1])*U(z, mu[j])*sqrt(lam[1])/sqrt(h(z))
    ),
    Eq(
        vhat(z, mu[j]),
        2*I*sqrt(-gamma[j] + lam[1])*V(z, mu[j])*sqrt(lam[1])/sqrt(h(z))
    )
]

UV_uv_j_h_subs = (
    [UV_to_uv_lam1_hs[0].subs(j, _j+1).args for _j in range(4)] + 
    [UV_to_uv_lam1_hs[1].subs(j, _j+1).args for _j in range(4)]
)
h_uv_sum = Eq(h(z), d[5] - Sum((gamma[l] - lam[1])*U(z, mu[l])*V(z, mu[l]), (l, 1, 4)))

h_uv_sum
UV_to_uv_lam1_hs[0]
UV_to_uv_lam1_hs[1]

Eq(h(z), d[5] - Sum((gamma[l] - lambda[1])*U(z, mu[l])*V(z, mu[l]), (l, 1, 4)))

Eq(uhat(z, mu[j]), 2*I*sqrt(-gamma[j] + lambda[1])*U(z, mu[j])*sqrt(lambda[1])/sqrt(h(z)))

Eq(vhat(z, mu[j]), 2*I*sqrt(-gamma[j] + lambda[1])*V(z, mu[j])*sqrt(lambda[1])/sqrt(h(z)))

We then derive an expression for $h(z)$ in terms of $\hat{u},\hat{v}$:

In [45]:
# Obtain expression for power in old modes in terms of new modes and h
UV_normed = Eq(
    UV_to_uv_lam1_hs[0].lhs*UV_to_uv_lam1_hs[1].lhs,
    UV_to_uv_lam1_hs[0].rhs*UV_to_uv_lam1_hs[1].rhs
)

# Sum that expression to replace new modes in terms of h 
UV_uv_normed_sum = Eq(
    Sum(UV_normed.lhs.subs(j,l), (l, 1, 4))/4,
    lam[1]*4*Sum((fraction(UV_normed.rhs.subs(j,l))[0]/lam[1]/4).factor(), (l, 1, 4))/(
        fraction(UV_normed.rhs.subs(j,l))[1]
    )/4
)
UV_uv_normed_sum_h = Eq(
    UV_uv_normed_sum.lhs, 
    UV_uv_normed_sum.rhs.subs(-h_uv_sum.rhs + d[5], -h_uv_sum.lhs + d[5])
)

# Isolate h in terms of old modes
UV_uv_normed_sum_invert = Eq(
    h(z),
    solve(Eq(x, UV_uv_normed_sum_h.rhs), h(z))[0].subs(x, UV_uv_normed_sum_h.lhs)
)

UV_normed
UV_uv_normed_sum
UV_uv_normed_sum_h
UV_uv_normed_sum_invert

Eq(uhat(z, mu[j])*vhat(z, mu[j]), -4*(-gamma[j] + lambda[1])*U(z, mu[j])*V(z, mu[j])*lambda[1]/h(z))

Eq(Sum(uhat(z, mu[l])*vhat(z, mu[l]), (l, 1, 4))/4, lambda[1]*Sum((gamma[l] - lambda[1])*U(z, mu[l])*V(z, mu[l]), (l, 1, 4))/h(z))

Eq(Sum(uhat(z, mu[l])*vhat(z, mu[l]), (l, 1, 4))/4, (-h(z) + d[5])*lambda[1]/h(z))

Eq(h(z), d[5]*lambda[1]/(lambda[1] + Sum(uhat(z, mu[l])*vhat(z, mu[l]), (l, 1, 4))/4))

We show now that $h$ can be expressed in terms of the power in any single one of the new modes, and that the new modes preserve intermodal power conservation but with new values for the associated constants:

In [46]:
# Start with power conservation of old modes but expressed in new modes and h
gamma_jk_uv_h = Eq(
    (UV_normed.lhs - UV_normed.lhs.subs(j,k))
    .subs([
        (uhat(z, mu[j])*vhat(z, mu[j]), gamma[j] - rho(z)),
        (uhat(z, mu[k])*vhat(z, mu[k]), gamma[k] - rho(z))
    ]),
    UV_normed.rhs - UV_normed.rhs.subs(j,k)
)

# Multiply by h and sum over k modes to anhiliate their contribution leaving expression in j modes
gamma_jk_uv_h_b = Eq(
    gamma_jk_uv_h.lhs*h(z),
    (gamma_jk_uv_h.rhs*h(z))
    .expand().collect([
        U(z, mu[j])*V(z, mu[j]), 
        U(z, mu[k])*V(z, mu[k])
    ], factor)
)
gamma_jk_uv_h_c = Eq(
    sum(Sum(_, (k,1,4))/4 for _ in gamma_jk_uv_h_b.lhs.expand().args),
    sum(Sum(_, (k,1,4))/4 for _ in gamma_jk_uv_h_b.rhs.args)
)

# Solve for h
_sum_term = 4*gamma_jk_uv_h_c.rhs.args[0]
_sum_term_summand = _sum_term.args[0]
_sum_term_limits = _sum_term.args[1]
gamma_jk_uv_h_d = Eq(
    gamma_jk_uv_h_c.lhs
    .doit().subs(gamma[4], solve(Eq(Sum(gamma[j],(j,1,4)).doit(),0), gamma[4])[0])
    .simplify(),
    gamma_jk_uv_h_c.rhs.subs([
        (gamma_jk_uv_h_c.rhs.args[1], gamma_jk_uv_h_c.rhs.args[1].doit()),
        (
            _sum_term,
            - lam[1]*4*Sum(
                -_sum_term_summand/lam[1]/4, 
                _sum_term_limits
            ).replace(k,l)
        ),
        (-h_uv_sum.rhs + d[5], -h_uv_sum.lhs + d[5])
    ])
)
gamma_jk_uv_h_e = Eq(
    h(z), solve(gamma_jk_uv_h_d, h(z))[0].expand().collect(d[5], factor)
)

# Get expression for power in mode j in terms of h
uvj_h = Eq(
    U(z, mu[j])*V(z, mu[j]), 
    solve(gamma_jk_uv_h_e, U(z, mu[j])*V(z, mu[j]))[0].expand().collect(h(z), factor)
)

# Get intermodal power conservation in new modes
UV_jk_pow_cons = Eq(
    (uvj_h.lhs - uvj_h.lhs.subs(j,k)),
    (uvj_h.rhs - uvj_h.rhs.subs(j,k))
)

gamma_jk_uv_h
gamma_jk_uv_h_b
gamma_jk_uv_h_c
gamma_jk_uv_h_e
uvj_h
UV_jk_pow_cons

Eq(gamma[j] - gamma[k], -4*(-gamma[j] + lambda[1])*U(z, mu[j])*V(z, mu[j])*lambda[1]/h(z) + 4*(-gamma[k] + lambda[1])*U(z, mu[k])*V(z, mu[k])*lambda[1]/h(z))

Eq((gamma[j] - gamma[k])*h(z), 4*(gamma[j] - lambda[1])*U(z, mu[j])*V(z, mu[j])*lambda[1] - 4*(gamma[k] - lambda[1])*U(z, mu[k])*V(z, mu[k])*lambda[1])

Eq(Sum(h(z)*gamma[j], (k, 1, 4))/4 + Sum(-h(z)*gamma[k], (k, 1, 4))/4, Sum(4*(gamma[j] - lambda[1])*U(z, mu[j])*V(z, mu[j])*lambda[1], (k, 1, 4))/4 + Sum(-4*(gamma[k] - lambda[1])*U(z, mu[k])*V(z, mu[k])*lambda[1], (k, 1, 4))/4)

Eq(h(z), 4*U(z, mu[j])*V(z, mu[j])*lambda[1] - d[5]*lambda[1]/(gamma[j] - lambda[1]))

Eq(U(z, mu[j])*V(z, mu[j]), h(z)/(4*lambda[1]) + d[5]/(4*(gamma[j] - lambda[1])))

Eq(U(z, mu[j])*V(z, mu[j]) - U(z, mu[k])*V(z, mu[k]), -d[5]/(4*(gamma[k] - lambda[1])) + d[5]/(4*(gamma[j] - lambda[1])))

It follows that:

In [47]:
dh_dUV_j = Eq(
    diff(h(z),z), 
    4*lam[1]*Derivative(U(z, mu[j])*V(z, mu[j]), z)    
)
dh_dUV_j_check = dh_dUV_j.subs([gamma_jk_uv_h_e.args]).doit().simplify()
if not dh_dUV_j_check:
    raise ValueError('dh_dUV_j_check failed')

dh_dUV_j

Eq(Derivative(h(z), z), 4*Derivative(U(z, mu[j])*V(z, mu[j]), z)*lambda[1])

We then obtain an expression for the derivative of $h$ in terms of $U,V$ starting from the differential equations in the old modes $\hat{u}, \hat{v}$, and thus subsequently obtain the expression for the derivative of power in mode $j$ which is seen to take the same form as before, i.e., it is invariant up to a constant:

In [55]:
# Get derivative of h in old modes
dUV_sum_dUV_j = Eq(
    Sum(uhat(z, mu[l])*Derivative(vhat(z, mu[l]), z) + vhat(z, mu[l])*Derivative(uhat(z, mu[l]), z), (l, 1, 4)),
    4*Derivative(uhat(z, mu[j])*vhat(z, mu[j]),z)
)
dh_lam1_UV = Eq(
    diff(UV_uv_normed_sum_invert.lhs,z), 
    diff(UV_uv_normed_sum_invert.rhs,z)
    .subs([dUV_sum_dUV_j.args, dUVj.args])
)

# Convert derivative of h to new modes
dh_lam1_uv = Eq(
    dh_lam1_UV.lhs, 
    (
        dh_lam1_UV.rhs * 
        UV_uv_normed_sum_invert.lhs**2/UV_uv_normed_sum_invert.rhs**2
    )
    .subs([
        UV_to_uv_lam1_hs[0].subs(j,_j+1).args for _j in range(4)
    ])
    .subs([
        UV_to_uv_lam1_hs[1].subs(j,_j+1).args for _j in range(4)
    ])
    .simplify()
    .subs(*C_gam_prod.doit().args)
)

# Get derivative of UV for j in terms of U,V
dUVj_d5 = Eq(dh_lam1_uv.lhs.subs([dh_dUV_j.args])/4/lam[1], dh_lam1_uv.rhs/4/lam[1])
dUVj_chi = dUVj_d5.subs([C_d5_chi.args])


dh_lam1_UV
dh_lam1_uv
dUVj_d5
dUVj_chi

Eq(Derivative(h(z), z), -(-uhat(z, mu[1])*uhat(z, mu[2])*uhat(z, mu[3])*uhat(z, mu[4]) + vhat(z, mu[1])*vhat(z, mu[2])*vhat(z, mu[3])*vhat(z, mu[4]))*d[5]*lambda[1]/(lambda[1] + Sum(uhat(z, mu[l])*vhat(z, mu[l]), (l, 1, 4))/4)**2)

Eq(Derivative(h(z), z), 16*C*(U(z, mu[1])*U(z, mu[2])*U(z, mu[3])*U(z, mu[4]) - V(z, mu[1])*V(z, mu[2])*V(z, mu[3])*V(z, mu[4]))*lambda[1]/d[5])

Eq(Derivative(U(z, mu[j])*V(z, mu[j]), z), 4*C*(U(z, mu[1])*U(z, mu[2])*U(z, mu[3])*U(z, mu[4]) - V(z, mu[1])*V(z, mu[2])*V(z, mu[3])*V(z, mu[4]))/d[5])

Eq(Derivative(U(z, mu[j])*V(z, mu[j]), z), chi*(U(z, mu[1])*U(z, mu[2])*U(z, mu[3])*U(z, mu[4]) - V(z, mu[1])*V(z, mu[2])*V(z, mu[3])*V(z, mu[4]))/(4*(-1)**m))

We then express $\hat{H}$ in $U,V$:

In [56]:
# Transform the product terms
Uu_h_prod = Eq(
    Product(uhat(z, mu[l]),(l,1,4)), 
    Product(
        uhat(z, mu[l])
        .subs(*UV_to_uv_lam1_hs[0].subs(j,l).args)/
        (U(z, mu[l]) * sqrt(lam[1] - gamma[l])),
        (l,1,4)
    ).simplify()*
    Product(
        U(z, mu[l]),
        (l,1,4)
    )*
    Product(
        sqrt(lam[1] - gamma[l]),
        (l,1,4)
    )
)
Vv_h_prod = Eq(
    Product(vhat(z, mu[l]),(l,1,4)), 
    Product(
        vhat(z, mu[l])
        .subs(*UV_to_uv_lam1_hs[1].subs(j,l).args)/
        (V(z, mu[l]) * sqrt(lam[1] - gamma[l])),
        (l,1,4)
    ).simplify()*
    Product(
        V(z, mu[l]),
        (l,1,4)
    )*
    Product(
        sqrt(lam[1] - gamma[l]),
        (l,1,4)
    )
)

# Transform the sum terms
UVuv_h_sum = Eq(
    Sum(
        uhat(z, mu[l])**2*vhat(z, mu[l])**2*b[2]/4 - 
        uhat(z, mu[l])*vhat(z, mu[l])*b[1]/4, 
        (l, 1, 4)
    ),
    Sum(
        ((uhat(z, mu[l])*vhat(z, mu[l]))**2)
        .subs([
            UV_to_uv_lam1_hs[0].subs(j,l).args,
            UV_to_uv_lam1_hs[1].subs(j,l).args
        ]).simplify()*b[2]/4 - 
        (uhat(z, mu[l])*vhat(z, mu[l]))
        .subs([
            UV_to_uv_lam1_hs[0].subs(j,l).args,
            UV_to_uv_lam1_hs[1].subs(j,l).args
        ]).simplify()*b[1]/4, 
        (l, 1, 4)
    )
)

# Combine the product and sum terms
K_uv_h = H_uv_hat.subs([
    Uu_h_prod.args,
    Vv_h_prod.args,
    UVuv_h_sum.args,
    C_gam_prod.args
])


H_uv_hat
K_uv_h

Eq(Hhat, Product(uhat(z, mu[l]), (l, 1, 4)) + Product(vhat(z, mu[l]), (l, 1, 4)) - Sum(uhat(z, mu[l])**2*vhat(z, mu[l])**2*b[2]/4 - uhat(z, mu[l])*vhat(z, mu[l])*b[1]/4, (l, 1, 4)))

Eq(Hhat, 16*C*lambda[1]**2*Product(U(z, mu[l]), (l, 1, 4))/h(z)**2 + 16*C*lambda[1]**2*Product(V(z, mu[l]), (l, 1, 4))/h(z)**2 - Sum(4*(gamma[l] - lambda[1])**2*U(z, mu[l])**2*V(z, mu[l])**2*b[2]*lambda[1]**2/h(z)**2 - (gamma[l] - lambda[1])*U(z, mu[l])*V(z, mu[l])*b[1]*lambda[1]/h(z), (l, 1, 4)))

The sum term can be converted to $h$, and $\hat{H}$ can be expressed in parameters. This leads to the relations:

In [57]:
UV_sum_arg_as_h = Eq(
    4*(gamma[l] - lam[1])**2*U(z, mu[l])**2*V(z, mu[l])**2*b[2]*lam[1]**2/h(z)**2 
    - (gamma[l] - lam[1])*U(z, mu[l])*V(z, mu[l])*b[1]*lam[1]/h(z),
    (
        4*(gamma[l] - lam[1])**2*U(z, mu[l])**2*V(z, mu[l])**2*b[2]*lam[1]**2/h(z)**2 
        - (gamma[l] - lam[1])*U(z, mu[l])*V(z, mu[l])*b[1]*lam[1]/h(z)
    ).subs([uvj_h.subs(j,l).args])
    .expand().collect([h(z), gamma[l]], factor)
)

UV_sum_as_h = Eq(
    Sum(
        4*(gamma[l] - lam[1])**2*U(z, mu[l])**2*V(z, mu[l])**2*b[2]*lam[1]**2/h(z)**2 - 
        (gamma[l] - lam[1])*U(z, mu[l])*V(z, mu[l])*b[1]*lam[1]/h(z)
        , 
        (l,1,4)
    ),
    (b[1] + b[2]*lam[1])*lam[1] 
    + (-b[1] - 2*b[2]*lam[1])*d[5]*lam[1]/h(z) 
    + b[2]*Sum(gamma[l]**2/4, (l,1,4))
    + b[2]*d[5]**2*lam[1]**2/h(z)**2
)

H_hat_h = K_uv_h.subs([UV_sum_as_h.args])
b0_hat_h = Eq(
    H_hat_h.lhs.subs([H_uv_hat_orig_gam.args]) + b[2]*Sum(gamma[l]**2/4, (l,1,4)) 
    + ((b[1]+b[2]*lam[1])*lam[1]).expand(),
    H_hat_h.rhs + b[2]*Sum(gamma[l]**2/4, (l,1,4)) + (b[1]+b[2]*lam[1])*lam[1]
).subs([C_gam_prod_sign.args])

H_hat_h
H_uv_hat_orig_gam
b0_hat_h

Eq(Hhat, 16*C*lambda[1]**2*Product(U(z, mu[l]), (l, 1, 4))/h(z)**2 + 16*C*lambda[1]**2*Product(V(z, mu[l]), (l, 1, 4))/h(z)**2 - (-b[1] - 2*b[2]*lambda[1])*d[5]*lambda[1]/h(z) - (b[1] + b[2]*lambda[1])*lambda[1] - b[2]*Sum(gamma[l]**2/4, (l, 1, 4)) - b[2]*d[5]**2*lambda[1]**2/h(z)**2)

Eq(Hhat, b[0] - b[2]*Sum(gamma[l]**2/4, (l, 1, 4)))

Eq(2*(-1)**m*C, 16*C*lambda[1]**2*Product(U(z, mu[l]), (l, 1, 4))/h(z)**2 + 16*C*lambda[1]**2*Product(V(z, mu[l]), (l, 1, 4))/h(z)**2 - (-b[1] - 2*b[2]*lambda[1])*d[5]*lambda[1]/h(z) - b[2]*d[5]**2*lambda[1]**2/h(z)**2)

We now calculate the differential equations for $U,V$ from the old ones for $\hat{u},\hat{v}$:

In [58]:
duj_b_h = dUj.subs([_.args for _ in UV_to_uv_lam1_hs]).subs(UV_uv_j_h_subs).doit()
dvj_b_h = dVj.subs([_.args for _ in UV_to_uv_lam1_hs]).subs(UV_uv_j_h_subs).doit()
duj_b_h = Eq(
    diff(U(z, mu[j]),z), 
    solve(duj_b_h, diff(U(z, mu[j]),z))[0]
    .expand().collect([diff(h(z),z), U(z, mu[j])], factor)
    .subs(*C_gam_prod.doit().args)
)
dvj_b_h = Eq(
    diff(V(z, mu[j]),z), 
    solve(dvj_b_h, diff(V(z, mu[j]),z))[0]
    .expand().collect([diff(h(z),z), V(z, mu[j])], factor)
    .subs(*C_gam_prod.doit().args)
)

duj_b_h
dvj_b_h

Eq(Derivative(U(z, mu[j]), z), 4*C*V(z, mu[1])*V(z, mu[2])*V(z, mu[3])*V(z, mu[4])*lambda[1]/((gamma[j] - lambda[1])*V(z, mu[j])*h(z)) - 2*(gamma[j] - lambda[1])*U(z, mu[j])**2*V(z, mu[j])*b[2]*lambda[1]/h(z) + U(z, mu[j])*b[1]/4 + U(z, mu[j])*Derivative(h(z), z)/(2*h(z)))

Eq(Derivative(V(z, mu[j]), z), -4*C*U(z, mu[1])*U(z, mu[2])*U(z, mu[3])*U(z, mu[4])*lambda[1]/((gamma[j] - lambda[1])*U(z, mu[j])*h(z)) + 2*(gamma[j] - lambda[1])*U(z, mu[j])*V(z, mu[j])**2*b[2]*lambda[1]/h(z) - V(z, mu[j])*b[1]/4 + V(z, mu[j])*Derivative(h(z), z)/(2*h(z)))

We elliminate the derivative of $h$, then elliminate the product of the non-conjugate modes using the Hamiltonian to obtain:

In [59]:
U_prod_b0 = Eq(
    Product(U(z, mu[l]), (l,1,4)).doit(), 
    solve(b0_hat_h.doit(), Product(U(z, mu[l]), (l,1,4)).doit())[0]
)
V_prod_b0 = Eq(
    Product(V(z, mu[l]), (l,1,4)).doit(), 
    solve(b0_hat_h.doit(), Product(V(z, mu[l]), (l,1,4)).doit())[0]
)

dUj_can = duj_b_h.subs([dh_lam1_uv.args, U_prod_b0.args])
dUj_can = Eq(
    dUj_can.lhs, 
    dUj_can.rhs
    .expand().collect([V_prod_b0.lhs, U(z, mu[j])], factor)
    .subs([uvj_h.args])
    .expand().collect([V_prod_b0.lhs, U(z, mu[j])], factor)
    .subs([gamma_jk_uv_h_e.args])
    .expand().collect([V_prod_b0.lhs, U(z, mu[j])], factor)
    .subs([C_gam_prod_sign.args])
)
dVj_can = dvj_b_h.subs([dh_lam1_uv.args, V_prod_b0.args])
dVj_can = Eq(
    dVj_can.lhs, 
    dVj_can.rhs
    .expand().collect([U_prod_b0.lhs, V(z, mu[j])], factor)
    .subs([uvj_h.args])
    .expand().collect([U_prod_b0.lhs, V(z, mu[j])], factor)
    .subs([gamma_jk_uv_h_e.args])
    .expand().collect([U_prod_b0.lhs, V(z, mu[j])], factor)
    .subs([C_gam_prod_sign.args])
)

dUj_can = dUj_can.subs(
    dUj_can.rhs.coeff(U(z, mu[j]),1)*U(z, mu[j]),
    apart(dUj_can.rhs.coeff(U(z, mu[j]),1), gamma[j])*U(z, mu[j])
)
dVj_can = dVj_can.subs(
    dVj_can.rhs.coeff(V(z, mu[j]),1)*V(z, mu[j]),
    apart(dVj_can.rhs.coeff(V(z, mu[j]),1), gamma[j])*V(z, mu[j])
)

dUj_can
dVj_can

Eq(Derivative(U(z, mu[j]), z), 4*(-1)**m*C*U(z, mu[j])**2*V(z, mu[j])/d[5] - 4*C*V(z, mu[1])*V(z, mu[2])*V(z, mu[3])*V(z, mu[4])/(V(z, mu[j])*d[5]) + (-(-1)**m*C/(gamma[j] - lambda[1]) - b[1]/4 - b[2]*gamma[j]/2 - b[2]*lambda[1]/2)*U(z, mu[j]))

Eq(Derivative(V(z, mu[j]), z), -4*(-1)**m*C*U(z, mu[j])*V(z, mu[j])**2/d[5] + 4*C*U(z, mu[1])*U(z, mu[2])*U(z, mu[3])*U(z, mu[4])/(U(z, mu[j])*d[5]) + ((-1)**m*C/(gamma[j] - lambda[1]) + b[1]/4 + b[2]*gamma[j]/2 + b[2]*lambda[1]/2)*V(z, mu[j]))

In [60]:
# Define the power sum constant

UV_hat_power_sum = Eq(
    Sum(U(z, mu[j])*V(z, mu[j])*gamma[j], (j, 1, 4)),
    d[5] + d[5]/4*lam[1]*Sum(1/(gamma[j] - lam[1]), (j, 1, 4))
)

# Define the new Hamiltonian

H_hat_UVj_C = Eq(
    -(-1)**m*C*d[5]*Sum((gamma[j] - lam[1])**(-2), (j, 1, 4))/8 
    - (b[1] + 4*b[2]*lam[1])*d[5]*Sum(1/(gamma[j] - lam[1]), (j, 1, 4))/16 
    - 3*b[2]*d[5]/4,
    -4*C/d[5]*(Product(U(z, mu[l]), (l, 1, 4)) + Product(V(z, mu[l]), (l, 1, 4))) + 
    2*(-1)**m*C/d[5]*Sum(U(z, mu[j])**2*V(z, mu[j])**2, (j, 1, 4)) - 
    Sum(
        ((-1)**m*C/(gamma[j] - lam[1]) + b[1]/4 + b[2]*lam[1]/2 + b[2]*gamma[j]/2)*U(z, mu[j])*V(z, mu[j]),
        (j, 1, 4)
    )
)
# Check that the new Hamiltonian is conserved
    
if not diff(H_hat_UVj_C.rhs.doit(),z).subs([
    dUj_can.subs(j,_j+1).args for _j in range(4)
]).subs([
    dVj_can.subs(j,_j+1).args for _j in range(4)
]).simplify().subs(gamma[4], - gamma[1] - gamma[2] - gamma[3]).factor() == 0:
    raise ValueError('H_hat_UVj_C not conserved')
    
# Check that the new Hamiltonian is equivalent to the old expression

prod_term_U = Product(U(z, mu[l]),(l,1,4))
prod_term_V = Product(V(z, mu[l]),(l,1,4))
H_hat_UVj_C_prod_terms = Eq(
    prod_term_U + prod_term_V, 
    (solve(H_hat_UVj_C, prod_term_U)[0] + prod_term_V.doit())
    .subs([
        uvj_h.subs(j,_j+1).args for _j in range(4)
    ])
    .expand().collect(h(z), simplify)
)
b0_hat_h_prod_terms = Eq(
    prod_term_U + prod_term_V, 
    solve(b0_hat_h, prod_term_U)[0] + prod_term_V.doit()
)

H_hat_UVj_C_check_lhs = (
    (H_hat_UVj_C_prod_terms.lhs - b0_hat_h_prod_terms.lhs)
    .subs(gamma[4], -gamma[1]-gamma[2]-gamma[3]).simplify()
) == 0
H_hat_UVj_C_check_rhs = (
    (H_hat_UVj_C_prod_terms.rhs - b0_hat_h_prod_terms.rhs)
    .subs(gamma[4], -gamma[1]-gamma[2]-gamma[3]).simplify()
) == 0
if not (H_hat_UVj_C_check_rhs and H_hat_UVj_C_check_lhs):
    raise ValueError('H_hat_UVj_C_check failed')
    
# Define the new Hamiltonian as K

K_ham_UVj_C_as_params = Eq(K, H_hat_UVj_C.lhs)
K_ham_UVj_C = Eq(K, H_hat_UVj_C.rhs)

# Cconvert sums to params

K_ham_UVj_C_as_params_b = K_ham_UVj_C_as_params.subs([
    gam_sum_d5_1_C.replace(k,j).args,
    gam_sum_d5_2_C.replace(k,j).args
])
K_ham_UVj_C_as_params_b = Eq(
    K_ham_UVj_C_as_params_b.lhs,
    K_ham_UVj_C_as_params_b.rhs.expand().collect(C, factor)
    .subs([
        ((-1)**(2*m),1)
    ])
)


# Check the new Hamiltonian produces the EOMs

for _j in range(4):
    if (
        diff(K_ham_UVj_C.rhs.doit(), V(z, mu[_j+1])) 
        - dUj_can.rhs.subs(j, _j+1)
    ).doit().subs(gamma[4], - gamma[1] - gamma[2] - gamma[3]).simplify() !=0:
        raise ValueError('H_hat_UVj_C EOMs different in U')
        
for _j in range(4):
    if (
        - diff(K_ham_UVj_C.rhs.doit(), U(z, mu[_j+1]))
        - dVj_can.rhs.subs(j, _j+1)
    ).doit().subs(gamma[4], - gamma[1] - gamma[2] - gamma[3]).simplify() !=0:
        raise ValueError('H_hat_UVj_C EOMs different in V')
    
    
UV_hat_power_sum
K_ham_UVj_C_as_params
K_ham_UVj_C_as_params_b
K_ham_UVj_C

Eq(Sum(U(z, mu[j])*V(z, mu[j])*gamma[j], (j, 1, 4)), d[5]*lambda[1]*Sum(1/(gamma[j] - lambda[1]), (j, 1, 4))/4 + d[5])

Eq(K, -(-1)**m*C*d[5]*Sum((gamma[j] - lambda[1])**(-2), (j, 1, 4))/8 - (b[1] + 4*b[2]*lambda[1])*d[5]*Sum(1/(gamma[j] - lambda[1]), (j, 1, 4))/16 - 3*b[2]*d[5]/4)

Eq(K, (-1)**m*(8*b[0]*b[2] - b[1]**2 + 6*b[1]*b[2]*lambda[1] + 8*b[2]**2*lambda[1]**2 + 4*c[2] + 24*lambda[1]**2)*d[5]/(16*C) - 7*b[2]*d[5]/4 + b[1]*d[5]**2/(64*C**2))

Eq(K, 2*(-1)**m*C*Sum(U(z, mu[j])**2*V(z, mu[j])**2, (j, 1, 4))/d[5] - 4*C*(Product(U(z, mu[l]), (l, 1, 4)) + Product(V(z, mu[l]), (l, 1, 4)))/d[5] - Sum(((-1)**m*C/(gamma[j] - lambda[1]) + b[1]/4 + b[2]*gamma[j]/2 + b[2]*lambda[1]/2)*U(z, mu[j])*V(z, mu[j]), (j, 1, 4)))

The coefficients in front of SPM and FWM terms have been equalised. The new phase velocity constants also sum to the reciprocal of that constant which encourages another gauge transform.

### Linear gauge transform to $\bar{u}, \bar{v}$
We start with the definition of the current modes $U, V$ in terms of the new modes $\bar{u}, \bar{v}$, related by a linear gauge transform:

In [61]:
phase_coeff_sum = Eq(
    Sum(-(-1)**m*C/(gamma[j] - lam[1]) - b[1]/4 - b[2]*gamma[j]/2 - b[2]*lam[1]/2, (j,1,4)),
    -(-1)**m*C*Sum(1/(gamma[j] - lam[1]),(j,1,4)) - b[1] - b[2]*lam[1]*2
)

d5_gam_lam1_C_b = Eq(
    -(d5_gam_lam1_C.rhs/C/4/(-1)**m).subs(C**2, (-1)**(2*m)*C**2).expand().collect(C, factor), 
    -d5_gam_lam1_C.lhs/C/4/(-1)**m
)

phase_coeff_sum = phase_coeff_sum.subs([d5_gam_lam1_C_b.replace(k,j).args])

bar_guage_shift = dUj_can.rhs.coeff(U(z, mu[j]))
ubar_gauge = Eq(
    U(z, mu[j]), 
    ubar(z, mu[j])*exp(z*(bar_guage_shift - phase_coeff_sum.rhs/4))
)
vbar_gauge = Eq(
    V(z, mu[j]), 
    vbar(z, mu[j])*exp(-z*(bar_guage_shift - phase_coeff_sum.rhs/4))
)


phase_coeff_sum
ubar_gauge
vbar_gauge

Eq(Sum(-(-1)**m*C/(gamma[j] - lambda[1]) - b[1]/4 - b[2]*gamma[j]/2 - b[2]*lambda[1]/2, (j, 1, 4)), -d[5]/(4*(-1)**m*C))

Eq(U(z, mu[j]), ubar(z, mu[j])*exp(z*(-(-1)**m*C/(gamma[j] - lambda[1]) - b[1]/4 - b[2]*gamma[j]/2 - b[2]*lambda[1]/2 + d[5]/(16*(-1)**m*C))))

Eq(V(z, mu[j]), vbar(z, mu[j])*exp(-z*(-(-1)**m*C/(gamma[j] - lambda[1]) - b[1]/4 - b[2]*gamma[j]/2 - b[2]*lambda[1]/2 + d[5]/(16*(-1)**m*C))))

In [71]:
# For the write up we use chi
phase_coeff_sum.subs([(d[5], solve(C_d5_chi,d[5])[0]), (C, solve(C_gam_prod_sign,C)[0])])
ubar_gauge.subs([(d[5], solve(C_d5_chi,d[5])[0]), (C, solve(C_gam_prod_sign,C)[0])])
vbar_gauge.subs([(d[5], solve(C_d5_chi,d[5])[0]), (C, solve(C_gam_prod_sign,C)[0])])
C_d5_chi
chi_d5_b

Eq(Sum(-b[1]/4 - b[2]*gamma[j]/2 - b[2]*lambda[1]/2 - (b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)/(2*(gamma[j] - lambda[1])), (j, 1, 4)), -4/chi)

Eq(U(z, mu[j]), ubar(z, mu[j])*exp(z*(-b[1]/4 - b[2]*gamma[j]/2 - b[2]*lambda[1]/2 - (b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)/(2*(gamma[j] - lambda[1])) + 1/chi)))

Eq(V(z, mu[j]), vbar(z, mu[j])*exp(-z*(-b[1]/4 - b[2]*gamma[j]/2 - b[2]*lambda[1]/2 - (b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)/(2*(gamma[j] - lambda[1])) + 1/chi)))

Eq(C, chi*d[5]/(16*(-1)**m))

Eq(chi, (8*b[0] + 8*b[1]*lambda[1] + 8*b[2]*lambda[1]**2)/d[5])

There is then only a single parameter $\chi$ in the differential equations for $\bar{u}, \bar{v}$:

In [81]:
u_v_bar_subs = [ubar_gauge.args, vbar_gauge.args]
U_to_ubar_prod = Eq(
    Product(U(z, mu[j]), (j,1,4)).doit(), 
    (
        Product(U(z, mu[j]), (j,1,4)).doit()
        .subs([ubar_gauge.subs(j, _j+1).args for _j in range(4)])
        *exp(phase_coeff_sum.rhs*z)*exp(-phase_coeff_sum.lhs*z)
    ).expand().simplify().expand()
)
V_to_vbar_prod = Eq(
    Product(V(z, mu[j]), (j,1,4)).doit(), 
    (
        Product(V(z, mu[j]), (j,1,4)).doit()
        .subs([vbar_gauge.subs(j, _j+1).args for _j in range(4)])
        *exp(-phase_coeff_sum.rhs*z)*exp(phase_coeff_sum.lhs*z)
    ).expand().simplify().expand()
)
u_v_bar_subs += [ubar_gauge.args, vbar_gauge.args, U_to_ubar_prod.args, V_to_vbar_prod.args]

du_bar = Eq(
    diff(ubar(z, mu[j]),z), 
    solve(dUj_can.subs(u_v_bar_subs).doit(), diff(ubar(z, mu[j]),z))[0]
    .simplify().expand()
)
dv_bar = Eq(
    diff(vbar(z, mu[j]),z), 
    solve(dVj_can.subs(u_v_bar_subs).doit(), diff(vbar(z, mu[j]),z))[0]
    .simplify().expand()
)

# For the paper
du_bar_chi = du_bar.subs([(C, solve(C_d5_chi,C)[0]),(m,2)])
dv_bar_chi = dv_bar.subs([(C, solve(C_d5_chi,C)[0]),(m,2)])

du_bar
dv_bar

du_bar_chi
dv_bar_chi

Eq(Derivative(ubar(z, mu[j]), z), 4*(-1)**m*C*ubar(z, mu[j])**2*vbar(z, mu[j])/d[5] - 4*C*vbar(z, mu[1])*vbar(z, mu[2])*vbar(z, mu[3])*vbar(z, mu[4])/(vbar(z, mu[j])*d[5]) - ubar(z, mu[j])*d[5]/(16*(-1)**m*C))

Eq(Derivative(vbar(z, mu[j]), z), -4*(-1)**m*C*ubar(z, mu[j])*vbar(z, mu[j])**2/d[5] + 4*C*ubar(z, mu[1])*ubar(z, mu[2])*ubar(z, mu[3])*ubar(z, mu[4])/(ubar(z, mu[j])*d[5]) + vbar(z, mu[j])*d[5]/(16*(-1)**m*C))

Eq(Derivative(ubar(z, mu[j]), z), chi*ubar(z, mu[j])**2*vbar(z, mu[j])/4 - chi*vbar(z, mu[1])*vbar(z, mu[2])*vbar(z, mu[3])*vbar(z, mu[4])/(4*vbar(z, mu[j])) - ubar(z, mu[j])/chi)

Eq(Derivative(vbar(z, mu[j]), z), chi*ubar(z, mu[1])*ubar(z, mu[2])*ubar(z, mu[3])*ubar(z, mu[4])/(4*ubar(z, mu[j])) - chi*ubar(z, mu[j])*vbar(z, mu[j])**2/4 + vbar(z, mu[j])/chi)

In [82]:
# Transform the old Hamiltonian

U_to_ubar_prod_uneval = Eq(Product(U(z, mu[l]), (l,1,4)), Product(ubar(z, mu[l]), (l,1,4)))
V_to_vbar_prod_uneval = Eq(Product(V(z, mu[l]), (l,1,4)), Product(vbar(z, mu[l]), (l,1,4)))
U_to_ubar_prod_uneval_check = U_to_ubar_prod_uneval.doit().subs([U_to_ubar_prod.args]).simplify()
V_to_vbar_prod_uneval_check = V_to_vbar_prod_uneval.doit().subs([V_to_vbar_prod.args]).simplify()

if not U_to_ubar_prod_uneval_check:
    raise ValueError('U_to_ubar_prod_uneval_check failed')
if not V_to_vbar_prod_uneval_check:
    raise ValueError('V_to_vbar_prod_uneval_check failed')

K_ham_uv_bar_j = (
    K_ham_UVj_C
    .subs([U_to_ubar_prod_uneval.args, V_to_vbar_prod_uneval.args])
    .replace(*ubar_gauge.args)
    .replace(*vbar_gauge.args)
)

# Obtain the new Hamiltonian with a bit of guess work

H_bar_uv_j = Eq(
    Hbar,
    2*(-1)**m*C*Sum(ubar(z, mu[j])**2*vbar(z, mu[j])**2, (j, 1, 4))/d[5] 
    - 4*C*(Product(ubar(z, mu[l]), (l, 1, 4)) + Product(vbar(z, mu[l]), (l, 1, 4)))/d[5] 
    - (-1)**m*d[5]/C/16*Sum(ubar(z, mu[j])*vbar(z, mu[j]), (j, 1, 4))
)

# Subtract the old Hamiltonian from the new to obtain an expression for the constant in params

H_bar_min_K_bar = Eq(
    H_bar_uv_j.lhs - K_ham_uv_bar_j.lhs,
    H_bar_uv_j.rhs - K_ham_uv_bar_j.rhs
)

_sum_arg_term_uvbar = (
    ((-1)**m*C/(gamma[j] - lam[1]) + b[1]/4 + b[2]*gamma[j]/2 + b[2]*lam[1]/2)*ubar(z, mu[j])*vbar(z, mu[j])
)
H_bar_min_K_bar_val_h = (
    H_bar_min_K_bar
    .replace(*uvj_h.subs(u_v_bar_subs).args)
    .replace(_sum_arg_term_uvbar, _sum_arg_term_uvbar.subs(*uvj_h.subs(u_v_bar_subs).args))
)


H_bar_min_K_bar_val = H_bar_min_K_bar_val_h.subs(h(z),0)

H_bar_min_K_bar_val_check = (
    (H_bar_min_K_bar_val.rhs - H_bar_min_K_bar_val_h.rhs)
    .doit().expand()
    .subs([d5_gam_lam1_C.args])
    .doit().expand()
    .subs([((-1)**(2*m), 1)])
    .simplify()
    .subs(gamma[4], - gamma[1] - gamma[2] - gamma[3])
) == 0
if not H_bar_min_K_bar_val_check:
    raise ValueError('H_bar_min_K_bar_val_check failed')
    



# Substitute sums to forms that have known evaluations
HK_bar_sum_1 = Eq(
    Sum(d[5]/(4*gamma[j] - 4*lam[1]), (j, 1, 4)),
    d[5]/4*Sum(1/(gamma[j] - lam[1]), (j, 1, 4))
)
if not (HK_bar_sum_1.lhs - HK_bar_sum_1.rhs).doit().expand().simplify() == 0:
    raise ValueError('HK_bar_sum_1 check failed')
    
HK_bar_sum_2 = Eq(
    Sum(
        ((-1)**m*C/(gamma[j] - lam[1]) + b[1]/4 + b[2]*gamma[j]/2 + b[2]*lam[1]/2)*d[5]/
        (4*gamma[j] - 4*lam[1]), 
        (j, 1, 4)
    ),
    (-1)**m*C*d[5]/4*Sum(1/(gamma[j] - lam[1])**2, (j, 1, 4)) +
    (b[1] + 2*b[2]*lam[1])*d[5]/16*Sum(1/(gamma[j] - lam[1]), (j, 1, 4)) +
    b[2]*d[5]/2 +
    b[2]*d[5]*lam[1]/8*Sum(1/(gamma[j] - lam[1]), (j, 1, 4))
)
if not (HK_bar_sum_2.lhs - HK_bar_sum_2.rhs).doit().expand().simplify() == 0:
    raise ValueError('HK_bar_sum_2 check failed')
    

H_bar_min_K_bar_val_b = H_bar_min_K_bar_val.subs([
    HK_bar_sum_1.args,
    HK_bar_sum_2.args,
    gam_sum_d5_1_C.replace(k,j).args,
    gam_sum_d5_2_C.replace(k,j).args
])
H_bar_min_K_bar_val_b = Eq(
    H_bar_min_K_bar_val_b.lhs,
    H_bar_min_K_bar_val_b.rhs
    .expand().collect(C, factor).subs([
        ((-1)**(2*m), 1)
    ])
)

H_bar_val = Eq(
    H_bar_min_K_bar_val_b.lhs + K,
    (H_bar_min_K_bar_val_b.rhs + K).subs([
        K_ham_UVj_C_as_params_b.args
    ]).expand().collect(C, factor)
)

# Check that the new Hamiltonian is conserved
    
if not (
    diff(H_bar_uv_j.rhs.doit(),z).subs([
        du_bar.subs(j,_j+1).args for _j in range(4)
    ]).subs([
        dv_bar.subs(j,_j+1).args for _j in range(4)
    ]).simplify().subs(gamma[4], - gamma[1] - gamma[2] - gamma[3]).factor()
    .expand().simplify().subs([((-1)**(2*m), 1)])
) == 0:
    raise ValueError('H_bar_uv_j not conserved')
    
    
# Check the new Hamiltonian produces the EOMs

for _j in range(4):
    if (
        diff(H_bar_uv_j.rhs.doit(), vbar(z, mu[_j+1])) 
        - du_bar.rhs.subs(j, _j+1)
    ).doit().subs(gamma[4], - gamma[1] - gamma[2] - gamma[3]).simplify().subs([((-1)**(2*m), 1)]) !=0:
        raise ValueError('H_bar_uv_j EOMs different in ubar')
        
for _j in range(4):
    if (
        - diff(H_bar_uv_j.rhs.doit(), ubar(z, mu[_j+1]))
        - dv_bar.rhs.subs(j, _j+1)
    ).doit().subs(gamma[4], - gamma[1] - gamma[2] - gamma[3]).simplify().subs([((-1)**(2*m), 1)]) !=0:
        raise ValueError('H_bar_uv_j EOMs different in vbar')

        
# For the paper
H_bar_uv_j_chi = H_bar_uv_j.subs([(C, solve(C_d5_chi,C)[0]),(m,2)])


H_bar_uv_j
H_bar_val
H_bar_uv_j_chi

Eq(Hbar, 2*(-1)**m*C*Sum(ubar(z, mu[j])**2*vbar(z, mu[j])**2, (j, 1, 4))/d[5] - (-1)**m*d[5]*Sum(ubar(z, mu[j])*vbar(z, mu[j]), (j, 1, 4))/(16*C) - 4*C*(Product(ubar(z, mu[l]), (l, 1, 4)) + Product(vbar(z, mu[l]), (l, 1, 4)))/d[5])

Eq(Hbar, -(-1)**m*(4*b[0]*b[2] - b[1]**2 + 2*c[2] + 12*lambda[1]**2)*d[5]/(8*C) - (-1)**m*d[5]**3/(256*C**3) + 3*b[2]*d[5]/4 - (b[1] + 2*b[2]*lambda[1])*d[5]**2/(64*C**2))

Eq(Hbar, -chi*(Product(ubar(z, mu[l]), (l, 1, 4)) + Product(vbar(z, mu[l]), (l, 1, 4)))/4 + chi*Sum(ubar(z, mu[j])**2*vbar(z, mu[j])**2, (j, 1, 4))/8 - Sum(ubar(z, mu[j])*vbar(z, mu[j]), (j, 1, 4))/chi)

### Rescaling coordinates to $\tilde{u}, \tilde{v}$ and scaling the argument $z \rightarrow t$


Finally, after a rescaling of the coordinates and the variable we are able to obtain a parameter free system:

In [83]:
## Length rescaling identity
# Eq(t, x*z)
# Eq(diff(utilde(t, mu[1]),t), Derivative(utilde(t, mu[1]),z)/Derivative(t,z))
# Eq(diff(utilde(t, mu[1]),t), Derivative(utilde(x*z, mu[1]),z)/diff(x*z,z))

z_scale = (-1)**m*d[5]/16/C

ubar_to_utilde = Eq(ubar(z, mu[j]), exp(-I*pi*m/4)*d[5]/C/8*utilde(z_scale * z, mu[j]))
vbar_to_vtilde = Eq(vbar(z, mu[j]), exp(I*pi*m/4)*d[5]/C/8*vtilde(z_scale * z, mu[j]))
bar_to_tilde_subs = [ubar_to_utilde.args, vbar_to_vtilde.args]
bar_to_tilde_subs += [ubar_to_utilde.subs(j, _j+1).args for _j in range(4)]
bar_to_tilde_subs += [vbar_to_vtilde.subs(j, _j+1).args for _j in range(4)]

z_t_d5 = Eq(z, t/z_scale)

minus_one_subs = [
    ((-1)**(2*m),1), (exp(I*pi*m),(-1)**m), (exp(-I*pi*m),(-1)**m), ((-1)**(-m),(-1)**m), ((-1)**(2*m),1)
]

du_tilde = du_bar.subs(bar_to_tilde_subs).subs([z_t_d5.args]).doit()
du_tilde = Eq(
    diff(utilde(t, mu[j]),t), 
    (solve(du_tilde.doit(), diff(utilde(t, mu[j]),t))[0]).expand()
    .subs(minus_one_subs)
)
dv_tilde = dv_bar.subs(bar_to_tilde_subs).subs([z_t_d5.args]).doit()
dv_tilde = Eq(
    diff(vtilde(t, mu[j]),t), 
    (solve(dv_tilde.doit(), diff(vtilde(t, mu[j]),t))[0]).expand()
    .subs(minus_one_subs)
)

ubar_to_utilde
vbar_to_vtilde
z_t_d5
du_tilde
dv_tilde

# for the paper

ubar_to_utilde.subs([(C, solve(C_d5_chi,C)[0]),(m,2)])
vbar_to_vtilde.subs([(C, solve(C_d5_chi,C)[0]),(m,2)])
z_t_d5.subs([(C, solve(C_d5_chi,C)[0]),(m,2)])

Eq(ubar(z, mu[j]), utilde((-1)**m*z*d[5]/(16*C), mu[j])*exp(-I*pi*m/4)*d[5]/(8*C))

Eq(vbar(z, mu[j]), vtilde((-1)**m*z*d[5]/(16*C), mu[j])*exp(I*pi*m/4)*d[5]/(8*C))

Eq(z, 16*C*t/((-1)**m*d[5]))

Eq(Derivative(utilde(t, mu[j]), t), utilde(t, mu[j])**2*vtilde(t, mu[j]) - utilde(t, mu[j]) - vtilde(t, mu[1])*vtilde(t, mu[2])*vtilde(t, mu[3])*vtilde(t, mu[4])/vtilde(t, mu[j]))

Eq(Derivative(vtilde(t, mu[j]), t), utilde(t, mu[1])*utilde(t, mu[2])*utilde(t, mu[3])*utilde(t, mu[4])/utilde(t, mu[j]) - utilde(t, mu[j])*vtilde(t, mu[j])**2 + vtilde(t, mu[j]))

Eq(ubar(z, mu[j]), -2*I*utilde(z/chi, mu[j])/chi)

Eq(vbar(z, mu[j]), 2*I*vtilde(z/chi, mu[j])/chi)

Eq(z, chi*t)

In [84]:
# Transform the old Hamiltonian

ubar_to_utilde_prod = Eq(
    Product(ubar(z, mu[j]), (j,1,4)).doit(), 
    (
        Product(ubar(z, mu[j]), (j,1,4)).doit()
        .subs([ubar_to_utilde.subs(j, _j+1).args for _j in range(4)])
    ).expand().simplify().expand()
)
vbar_to_vtilde_prod = Eq(
    Product(vbar(z, mu[j]), (j,1,4)).doit(), 
    (
        Product(vbar(z, mu[j]), (j,1,4)).doit()
        .subs([vbar_to_vtilde.subs(j, _j+1).args for _j in range(4)])
    ).expand().simplify().expand()
)

tilde_prod_scale = d[5]**4/C**4/4096
ubar_to_utilde_prod_uneval = Eq(
    Product(ubar(z, mu[l]), (l,1,4)), 
    exp(-I*pi*m)*tilde_prod_scale*Product(utilde(z*z_scale, mu[l]), (l,1,4))
)
vbar_to_vtilde_prod_uneval = Eq(
    Product(vbar(z, mu[l]), (l,1,4)), 
    exp(I*pi*m)*tilde_prod_scale*Product(vtilde(z*z_scale, mu[l]), (l,1,4))
)
ubar_to_utilde_prod_uneval_check = ubar_to_utilde_prod_uneval.doit().subs([ubar_to_utilde_prod.args]).simplify()
vbar_to_vtilde_prod_uneval_check = vbar_to_vtilde_prod_uneval.doit().subs([vbar_to_vtilde_prod.args]).simplify()

if not ubar_to_utilde_prod_uneval_check:
    raise ValueError('ubar_to_utilde_prod_uneval_check failed')
if not vbar_to_vtilde_prod_uneval_check:
    raise ValueError('vbar_to_vtilde_prod_uneval_check failed')


tilde_sum_1 = Eq(
    Sum(utilde(t, mu[j])**2*vtilde(t, mu[j])**2*d[5]**4/(4096*C**4), (j, 1, 4)),
    d[5]**4/(4096*C**4)*Sum(utilde(t, mu[j])**2*vtilde(t, mu[j])**2, (j, 1, 4))
)
tilde_sum_2 = Eq(
    Sum(utilde(t, mu[j])*vtilde(t, mu[j])*d[5]**2/(64*C**2), (j, 1, 4)),
    d[5]**2/(64*C**2)*Sum(utilde(t, mu[j])*vtilde(t, mu[j]), (j, 1, 4))
)

H_tilde_uv_j = (
    H_bar_uv_j.subs([
        ubar_to_utilde_prod_uneval.args, 
        vbar_to_vtilde_prod_uneval.args,
        (exp(-I*pi*m), (-1)**m),
        (exp(I*pi*m), (-1)**m)
    ])
    .replace(*ubar_to_utilde.args)
    .replace(*vbar_to_vtilde.args)
    .subs([
        (z, solve(z_t_d5,z)[0]),
        tilde_sum_1.args,
        tilde_sum_2.args
    ])
)

tilde_ham_scale = 1024*C**3/d[5]**3/(-1)**m
H_tilde_uv_j = Eq(
    tilde_ham_scale*H_tilde_uv_j.lhs,
    (tilde_ham_scale*H_tilde_uv_j.rhs).expand().subs((-1)**(2*m),1)
)

H_tilde_uv_j_val = Eq(Htilde, H_tilde_uv_j.lhs)
H_tilde_uv_j = Eq(
    H_tilde_uv_j.lhs.subs(H_tilde_uv_j_val.rhs, H_tilde_uv_j_val.lhs), 
    H_tilde_uv_j.rhs
)


# Check that the new Hamiltonian is conserved
    
if not (
    diff(H_tilde_uv_j.rhs.doit(),t).subs([
        du_tilde.subs(j,_j+1).args for _j in range(4)
    ]).subs([
        dv_tilde.subs(j,_j+1).args for _j in range(4)
    ]).simplify().subs(gamma[4], - gamma[1] - gamma[2] - gamma[3]).factor()
    .expand().simplify().subs([((-1)**(2*m), 1)])
) == 0:
    raise ValueError('H_bar_uv_j not conserved')
    
    
# Check the new Hamiltonian produces the EOMs

for _j in range(4):
    if (
        diff(H_tilde_uv_j.rhs.doit(), vtilde(t, mu[_j+1])) 
        - du_tilde.rhs.subs(j, _j+1)
    ).doit().subs(gamma[4], - gamma[1] - gamma[2] - gamma[3]).simplify().subs([((-1)**(2*m), 1)]) !=0:
        raise ValueError('H_tilde_uv_j EOMs different in utilde')
        
for _j in range(4):
    if (
        - diff(H_tilde_uv_j.rhs.doit(), utilde(t, mu[_j+1]))
        - dv_tilde.rhs.subs(j, _j+1)
    ).doit().subs(gamma[4], - gamma[1] - gamma[2] - gamma[3]).simplify().subs([((-1)**(2*m), 1)]) !=0:
        raise ValueError('H_tilde_uv_j EOMs different in vtilde')


H_tilde_uv_j
H_tilde_uv_j_val

# For the paper
H_tilde_uv_j_val.subs([(C, solve(C_d5_chi,C)[0]),(m,2)])

Eq(Htilde, -Product(utilde(t, mu[l]), (l, 1, 4)) - Product(vtilde(t, mu[l]), (l, 1, 4)) - Sum(utilde(t, mu[j])*vtilde(t, mu[j]), (j, 1, 4)) + Sum(utilde(t, mu[j])**2*vtilde(t, mu[j])**2, (j, 1, 4))/2)

Eq(Htilde, 1024*C**3*Hbar/((-1)**m*d[5]**3))

Eq(Htilde, Hbar*chi**3/4)

In [85]:
H_tilde_uv_j_val_b = Eq(
    H_tilde_uv_j_val.lhs,
    H_tilde_uv_j_val.rhs.subs([H_bar_val.args])
    .expand().collect(d[5], factor)
)

H_tilde_sum_1 = Eq(
    (-16*C*(b[1] + 2*b[2]*lam[1])/((-1)**m*d[5])),
    (-16*C*(b[1] + 2*b[2]*lam[1])/((-1)**m*d[5])).subs([
        (C, solve(C_gam_prod_sign, C)[0])
    ]).subs([
        ((-1)**(2*m),1), ((-1)**(-4*m),1), ((-1)**(4*m+1),-1)
    ])
)

H_tilde_sum_2 = Eq(
    -128*C**2*(
        4*(-1)**m*b[0]*b[2] - (-1)**m*b[1]**2 + 2*(-1)**m*c[2] + 12*(-1)**m*lam[1]**2 - 6*C*b[2]
    )/((-1)**m*d[5]**2),
    (-128*C**2*(
        4*(-1)**m*b[0]*b[2] - (-1)**m*b[1]**2 + 2*(-1)**m*c[2] + 12*(-1)**m*lam[1]**2 - 6*C*b[2]
    )/((-1)**m*d[5]**2)).subs([
        (C, solve(C_gam_prod_sign, C)[0])
    ]).factor().subs([
        ((-1)**(2*m),1), ((-1)**(-4*m),1), ((-1)**(4*m+1),-1)
    ])
)

H_tilde_uv_j_val_c = H_tilde_uv_j_val_b.subs([
    H_tilde_sum_1.args,
    H_tilde_sum_2.args
]) 

H_tilde_uv_j_val_c

Eq(Htilde, -8*(b[1] + 2*b[2]*lambda[1])*(b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)/d[5] - 32*(b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)**2*(b[0]*b[2] - b[1]**2 - 3*b[1]*b[2]*lambda[1] - 3*b[2]**2*lambda[1]**2 + 2*c[2] + 12*lambda[1]**2)/d[5]**2 - 4)

## Solutions in the hat system

The aim has been to show that we can obtain a canonical coordinate system for four wave mixing in which the modes are single valued meromorphic functions, namely Kronecker theta functions (ratios of two Weierstrass sigma functions which are themselves entire functions). We proceed to show that the coordinate transformations have delivered that by solving the differential equations in the $\hat{u},\hat{v}$ coordinates.

To start, we recall the system:

In [86]:
hz_UV_uvbar = gamma_jk_uv_h_e.subs([ubar_gauge.args, vbar_gauge.args])
uvbar_hz = uvj_h.subs([ubar_gauge.args, vbar_gauge.args])


du_bar
dv_bar
H_bar_uv_j
H_bar_val

hz_UV_uvbar
uvbar_hz

Eq(Derivative(ubar(z, mu[j]), z), 4*(-1)**m*C*ubar(z, mu[j])**2*vbar(z, mu[j])/d[5] - 4*C*vbar(z, mu[1])*vbar(z, mu[2])*vbar(z, mu[3])*vbar(z, mu[4])/(vbar(z, mu[j])*d[5]) - ubar(z, mu[j])*d[5]/(16*(-1)**m*C))

Eq(Derivative(vbar(z, mu[j]), z), -4*(-1)**m*C*ubar(z, mu[j])*vbar(z, mu[j])**2/d[5] + 4*C*ubar(z, mu[1])*ubar(z, mu[2])*ubar(z, mu[3])*ubar(z, mu[4])/(ubar(z, mu[j])*d[5]) + vbar(z, mu[j])*d[5]/(16*(-1)**m*C))

Eq(Hbar, 2*(-1)**m*C*Sum(ubar(z, mu[j])**2*vbar(z, mu[j])**2, (j, 1, 4))/d[5] - (-1)**m*d[5]*Sum(ubar(z, mu[j])*vbar(z, mu[j]), (j, 1, 4))/(16*C) - 4*C*(Product(ubar(z, mu[l]), (l, 1, 4)) + Product(vbar(z, mu[l]), (l, 1, 4)))/d[5])

Eq(Hbar, -(-1)**m*(4*b[0]*b[2] - b[1]**2 + 2*c[2] + 12*lambda[1]**2)*d[5]/(8*C) - (-1)**m*d[5]**3/(256*C**3) + 3*b[2]*d[5]/4 - (b[1] + 2*b[2]*lambda[1])*d[5]**2/(64*C**2))

Eq(h(z), 4*ubar(z, mu[j])*vbar(z, mu[j])*lambda[1] - d[5]*lambda[1]/(gamma[j] - lambda[1]))

Eq(ubar(z, mu[j])*vbar(z, mu[j]), h(z)/(4*lambda[1]) + d[5]/(4*(gamma[j] - lambda[1])))

We will first solve a differential equation for $h$. To do so we first convert the sums in the Hamiltonian to expressions in $h$:

In [87]:
# Convert sum terms to h

uvbar_hz_sum = Eq(
    Sum(ubar(z, mu[j])*vbar(z, mu[j]),(j,1,4)),
    h(z)/lam[1] + d[5]/4*Sum(1/(gamma[j] - lam[1]),(j,1,4))
)

uvbar_hz_sum_check = (Sum(uvbar_hz.rhs, (j,1,4)) - uvbar_hz_sum.rhs).doit().simplify() == 0
if not uvbar_hz_sum_check:
    raise ValueError('uvbar_hz_sum_check failed')

uvbar_hz_sqrd = Eq(uvbar_hz.lhs**2, (uvbar_hz.rhs**2).expand().collect(h(z), factor))
uvbar_hz_sqrd_sum = Eq(
    Sum(ubar(z, mu[j])**2*vbar(z, mu[j])**2,(j,1,4)),
    h(z)**2/(4*lam[1]**2) + 
    h(z)*d[5]/lam[1]/8*Sum(1/(gamma[j] - lam[1]),(j,1,4)) + 
    d[5]**2/16*Sum(1/(gamma[j] - lam[1])**2,(j,1,4))
)

uvbar_hz_sqrd_sum_check = (Sum(uvbar_hz_sqrd.rhs, (j,1,4)) - uvbar_hz_sqrd_sum.rhs).doit().simplify() == 0
if not uvbar_hz_sqrd_sum_check:
    raise ValueError('uvbar_hz_sqrd_sum_check failed')

gam_sum_d5_1_C_subs = [
    gam_sum_d5_1_C.replace(k,j).args,
    gam_sum_d5_2_C.replace(k,j).args
]
gam_sum_d5_1_subs = [
    gam_sum_d5_1.replace(k,j).args,
    gam_sum_d5_2.replace(k,j).args
]

uvbar_hz_sum_b = uvbar_hz_sum.subs(gam_sum_d5_1_subs)
uvbar_hz_sqrd_sum_b = uvbar_hz_sqrd_sum.subs(gam_sum_d5_1_subs)
uvbar_hz_sum_C = uvbar_hz_sum.subs(gam_sum_d5_1_C_subs)
uvbar_hz_sqrd_sum_C = uvbar_hz_sqrd_sum.subs(gam_sum_d5_1_C_subs)

# Sub in sums in term of h into H

H_bar_uv_j_h = H_bar_uv_j.subs([
    uvbar_hz_sum_C.args,
    uvbar_hz_sqrd_sum_C.args
])
H_bar_uv_j_h = Eq(
    H_bar_uv_j_h.lhs, 
    H_bar_uv_j_h.rhs.expand().collect([h(z), d[5]], factor)
)



uvbar_hz_sum
uvbar_hz_sqrd_sum
# uvbar_hz_sum_b
# uvbar_hz_sqrd_sum_b
uvbar_hz_sum_C
uvbar_hz_sqrd_sum_C

H_bar_uv_j_h

Eq(Sum(ubar(z, mu[j])*vbar(z, mu[j]), (j, 1, 4)), h(z)/lambda[1] + d[5]*Sum(1/(gamma[j] - lambda[1]), (j, 1, 4))/4)

Eq(Sum(ubar(z, mu[j])**2*vbar(z, mu[j])**2, (j, 1, 4)), h(z)**2/(4*lambda[1]**2) + h(z)*d[5]*Sum(1/(gamma[j] - lambda[1]), (j, 1, 4))/(8*lambda[1]) + d[5]**2*Sum((gamma[j] - lambda[1])**(-2), (j, 1, 4))/16)

Eq(Sum(ubar(z, mu[j])*vbar(z, mu[j]), (j, 1, 4)), h(z)/lambda[1] + (-2*(-1)**m*C*(2*b[1] + 4*b[2]*lambda[1]) + d[5])*d[5]/(16*C**2))

Eq(Sum(ubar(z, mu[j])**2*vbar(z, mu[j])**2, (j, 1, 4)), (8*(-1)**m*b[2]/C - (-1)**m*(b[1] + 2*b[2]*lambda[1])*d[5]/(4*C**3) + (-4*b[0]*b[2] + b[1]**2 - 2*c[2] - 12*lambda[1]**2)/C**2)*d[5]**2/16 + h(z)**2/(4*lambda[1]**2) + (-2*(-1)**m*C*(2*b[1] + 4*b[2]*lambda[1]) + d[5])*h(z)*d[5]/(32*C**2*lambda[1]))

Eq(Hbar, -(-1)**(2*m)*(b[1] + 2*b[2]*lambda[1])*h(z)/(4*lambda[1]) - (-1)**(2*m)*(b[1] + 2*b[2]*lambda[1])*d[5]**2/(64*C**2) + (-1)**m*C*h(z)**2/(2*d[5]*lambda[1]**2) + (-1)**m*(8*(-1)**m*C*b[2] - 4*b[0]*b[2] + b[1]**2 - 2*c[2] - 12*lambda[1]**2)*d[5]/(8*C) - (-1)**m*d[5]**3/(256*C**3) - 4*C*(Product(ubar(z, mu[l]), (l, 1, 4)) + Product(vbar(z, mu[l]), (l, 1, 4)))/d[5])

When the constant terms from the rhs are subtracted from both sides, $\hat{H}$ can be replaced with a simple term $-b_2 d_5/4$. It is reassuring to observe such simplification after lengthy algebraic transformations.

In [88]:
const_term_from_h = H_bar_uv_j_h.rhs.replace(l,j).subs([(h(z),0), (ubar(z, mu[j]),0), (vbar(z, mu[j]),0)]).doit()
H_bar_val_min_h_const = Eq(
    (H_bar_val.lhs - const_term_from_h),
    (H_bar_val.rhs - const_term_from_h).expand().collect(C,factor).expand().subs([((-1)**(2*m),1)])
)

H_bar_uv_j_h_b = Eq(
    (H_bar_uv_j_h.lhs - const_term_from_h).subs([H_bar_val_min_h_const.args]),
    (H_bar_uv_j_h.rhs - const_term_from_h).subs([((-1)**(2*m),1)])
)


H_bar_val_min_h_const
H_bar_uv_j_h_b

Eq((-1)**(2*m)*(b[1] + 2*b[2]*lambda[1])*d[5]**2/(64*C**2) - (-1)**m*(8*(-1)**m*C*b[2] - 4*b[0]*b[2] + b[1]**2 - 2*c[2] - 12*lambda[1]**2)*d[5]/(8*C) + (-1)**m*d[5]**3/(256*C**3) + Hbar, -b[2]*d[5]/4)

Eq(-b[2]*d[5]/4, (-1)**m*C*h(z)**2/(2*d[5]*lambda[1]**2) - 4*C*(Product(ubar(z, mu[l]), (l, 1, 4)) + Product(vbar(z, mu[l]), (l, 1, 4)))/d[5] - (b[1] + 2*b[2]*lambda[1])*h(z)/(4*lambda[1]))

We likewise convert the product of modal powers to a polynomial in $h$:

In [89]:
H_bar_val_b_prods = Eq(
    H_bar_uv_j_h_b.lhs - H_bar_uv_j_h_b.lhs - H_bar_uv_j_h_b.rhs.subs(h(z),0),
    H_bar_uv_j_h_b.rhs - H_bar_uv_j_h_b.lhs - H_bar_uv_j_h_b.rhs.subs(h(z),0)
)

uv_bar_prod_h = Eq(
    Product(ubar(z,mu[j])*vbar(z, mu[j]), (j,1,4)),
    Product((ubar(z,mu[j])*vbar(z, mu[j])).subs([uvbar_hz.args]), (j,1,4))
)

gamma_lam_prod_as_C = Eq(
    C_gam_prod_sqrd.rhs.subs(gamma_j_prod_b_poly.rhs, gamma_j_prod_b_poly.lhs).doit(),
    C_gam_prod_sqrd.lhs
)
uv_bar_prod_h_poly =Eq(
    uv_bar_prod_h.lhs,
    uv_bar_prod_h.rhs.doit().expand().collect(h(z), factor).subs([
        gamma_lam_prod_as_C.args
    ])
    .expand().collect([lam[1], h(z)], factor)
    .subs([
        (sum([gamma[_j+1] for _j in range(4)]), 0),
        (-c_j_coeffs[1].rhs.doit(), -c_j_coeffs[1].lhs),
        (c_j_coeffs[2].rhs.doit().expand(), c_j_coeffs[2].lhs)
    ])
    .expand().collect(h(z), factor)
)

H_bar_val_b_prods
gamma_lam_prod_as_C
uv_bar_prod_h
uv_bar_prod_h_poly

Eq(4*C*(Product(ubar(z, mu[l]), (l, 1, 4)) + Product(vbar(z, mu[l]), (l, 1, 4)))/d[5], (-1)**m*C*h(z)**2/(2*d[5]*lambda[1]**2) - (b[1] + 2*b[2]*lambda[1])*h(z)/(4*lambda[1]) + b[2]*d[5]/4)

Eq((gamma[1] - lambda[1])*(gamma[2] - lambda[1])*(gamma[3] - lambda[1])*(gamma[4] - lambda[1]), C**2)

Eq(Product(ubar(z, mu[j])*vbar(z, mu[j]), (j, 1, 4)), Product(h(z)/(4*lambda[1]) + d[5]/(4*(gamma[j] - lambda[1])), (j, 1, 4)))

Eq(Product(ubar(z, mu[j])*vbar(z, mu[j]), (j, 1, 4)), h(z)**4/(256*lambda[1]**4) + (c[2] + 6*lambda[1]**2)*h(z)**2*d[5]**2/(256*C**2*lambda[1]**2) - (c[1] + 2*c[2]*lambda[1] + 4*lambda[1]**3)*h(z)**3*d[5]/(256*C**2*lambda[1]**3) - h(z)*d[5]**3/(64*C**2) + d[5]**4/(256*C**2))

We then construct the differential equation for the square of the derivative of $h$ and confirm there is no quartic term in the rhs. Again, this is by design and reassuring to observe:

In [90]:
duv_bar = Eq(
    Derivative(ubar(z, mu[j])*vbar(z, mu[j]),z), 
    Derivative(ubar(z, mu[j])*vbar(z, mu[j]),z).doit().subs([
        du_bar.args,
        dv_bar.args
    ])
    .expand().factor()
)
dh_uv_bar_sqrd = Eq(
    16*lam[1]**2*(duv_bar.lhs**2).subs([uvbar_hz.args]).doit(), 
    (
        16*lam[1]**2*(
            (duv_bar.rhs**2).expand()
            - ((H_bar_val_b_prods.lhs)**2).doit().expand() + H_bar_val_b_prods.rhs**2
        ).subs([
            uv_bar_prod_h_poly.doit().args
        ])
        .expand().subs([((-1)**(2*m),1)]).collect(h(z), factor)
        .subs([
            (C_gam_prod_sign.rhs/2, C_gam_prod_sign.lhs/2)
        ])
    ).expand().collect(h(z), factor)
)
dh_uv_bar_sqrd = Eq(
    dh_uv_bar_sqrd.lhs, 
    sum(_.collect(lam[1]) for _ in dh_uv_bar_sqrd.rhs.args)
)

to_d_subs = [
    (b[2]**2, solve(d_j_coeffs[4].subs([c_j_coeffs[4].args]), b[2]**2)[0]),
    (b[1]*b[2], solve(d_j_coeffs[3].subs([c_j_coeffs[3].args]), b[1]*b[2])[0]),
    (d_j_coeffs[2].rhs, d_j_coeffs[2].lhs),
    (d_j_coeffs[1].rhs, d_j_coeffs[1].lhs),
    (d_j_coeffs[1].rhs/2, d_j_coeffs[1].lhs/2),
    (d5_lam1.rhs, d5_lam1.lhs),
    d5_lam1.args
]

dh_uv_bar_sqrd_to_d = Eq(
    dh_uv_bar_sqrd.lhs, 
    dh_uv_bar_sqrd.rhs.subs(to_d_subs).collect(h(z),expand).subs(to_d_subs).collect(h(z),factor)
)

duv_bar
dh_uv_bar_sqrd_to_d

Eq(Derivative(ubar(z, mu[j])*vbar(z, mu[j]), z), 4*C*(ubar(z, mu[1])*ubar(z, mu[2])*ubar(z, mu[3])*ubar(z, mu[4]) - vbar(z, mu[1])*vbar(z, mu[2])*vbar(z, mu[3])*vbar(z, mu[4]))/d[5])

Eq(Derivative(h(z), z)**2, -(d[3] + 4*d[4]*lambda[1])*(d[1] + 2*d[2]*lambda[1] + 3*d[3]*lambda[1]**2 + 4*d[4]*lambda[1]**3)*h(z)*lambda[1] + (d[2] + 3*d[3]*lambda[1] + 6*d[4]*lambda[1]**2)*h(z)**2 + (d[1] + 2*d[2]*lambda[1] + 3*d[3]*lambda[1]**2 + 4*d[4]*lambda[1]**3)**2*d[4]*lambda[1]**2 - h(z)**3/lambda[1])

As always, we then introduce $w(z)$ to kill the quadratic term and make the cubic coefficient equal to $4$ so the differential equation resembles that of Weierstrass. In doing so, we confirm that the invariants $g_2, g_3$ are the same as those previously obtained, as indeed they should be:

In [91]:
h_to_w_eps = Eq(h(z), -4*w(z)*lam[1] + epsilon)
dw_bar = dh_uv_bar_sqrd_to_d.subs([h_to_w_eps.args]).doit()
dw_bar = Eq(dw_bar.lhs/16/lam[1]**2, (dw_bar.rhs/16/lam[1]**2).expand().collect(w(z), factor))
eps_sol = Eq(epsilon, solve(dw_bar.rhs.coeff(w(z),2),epsilon)[0])
dw_bar = Eq(
    dw_bar.lhs, 
    dw_bar.rhs.subs([eps_sol.args])
    .expand().collect(w(z), simplify)
    .subs(lam[1]**4, solve(lam1_root, lam[1]**4)[0])
    .expand().collect(w(z), simplify)
)
dw_bar_g2_g3 = dw_bar.subs([(g2_dj.rhs, g2_dj.lhs), (g3_dj.rhs, g3_dj.lhs)])
h_to_w = h_to_w_eps.subs([eps_sol.args])
w_wp = Eq(w(z), wp(z-z0, g2, g3))

h_to_w
dw_bar
g2_dj
g3_dj
dw_bar_g2_g3
w_wp

Eq(h(z), (d[2] + 3*d[3]*lambda[1] + 6*d[4]*lambda[1]**2)*lambda[1]/3 - 4*w(z)*lambda[1])

Eq(Derivative(w(z), z)**2, (-d[0]*d[4] + d[1]*d[3]/4 - d[2]**2/12)*w(z) + 4*w(z)**3 - d[0]*d[2]*d[4]/6 + d[0]*d[3]**2/16 + d[1]**2*d[4]/16 - d[1]*d[2]*d[3]/48 + d[2]**3/216)

Eq(g2, d[0]*d[4] - d[1]*d[3]/4 + d[2]**2/12)

Eq(g3, d[0]*d[2]*d[4]/6 - d[0]*d[3]**2/16 - d[1]**2*d[4]/16 + d[1]*d[2]*d[3]/48 - d[2]**3/216)

Eq(Derivative(w(z), z)**2, -g2*w(z) - g3 + 4*w(z)**3)

Eq(w(z), wp(z - z0, g2, g3))

In [92]:
print(latex(h_to_w))

h{\left(z \right)} = \frac{\left({d}_{2} + 3 {d}_{3} {\lambda}_{1} + 6 {d}_{4} {\lambda}_{1}^{2}\right) {\lambda}_{1}}{3} - 4 w{\left(z \right)} {\lambda}_{1}


We then use the solution for $h$ to get solutions for modal power $\hat{u}(z, \mu_j)\hat{v}(z, \mu_j)$ and we confirm that the constants that appear in those solutions are identical to the ones we would expect from the original coordinate system. We thus maintain the definition of points $z_1$ and $\mu_j - z_0$:

In [41]:
wp_z1_d_lam1 = Eq(wp(z1, g2, g3), d[2]/12 + d[3]*lam[1]/4 + d[4]*lam[1]**2/2)
wp_z0_mu_j_d = Eq(
    wp(-z0 + mu[j], g2, g3),
    d[2]/12 + d[3]*lam[1]/4 + d[4]*lam[1]**2/2 - 
    (-d[1] - 2*d[2]*lam[1] - 3*d[3]*lam[1]**2 - 4*d[4]*lam[1]**3)/(4*(gamma[j] - lam[1]))
)
wpp_z0_mu_j_d = Eq(
    wpp(-z0 + mu[j], g2, g3),
    -(b[0] + b[1]*gamma[j] + b[2]*gamma[j]**2)*
    (d[1] + 2*d[2]*lam[1] + 3*d[3]*lam[1]**2 + 4*d[4]*lam[1]**3)/(4*(gamma[j] - lam[1])**2)
)

hz_bar_wp = h_to_w.subs([w_wp.args, (d[4], solve(wp_z1_d_lam1,d[4])[0])])
uvbar_wp = uvbar_hz.subs([hz_bar_wp.args])
uvbar_wp_d5 = Eq(uvbar_wp.lhs, uvbar_wp.rhs.collect(d[5], factor))

uvbar_wp = Eq(
    uvbar_wp_d5.lhs,
    ((uvbar_wp_d5.rhs.subs([wp_z1_d_lam1.args, d5_lam1.args]) + wp_z0_mu_j_d.lhs - wp_z0_mu_j_d.rhs))
    .simplify()
)
duvbar_wpp = Eq(
    Derivative(uvbar_wp.lhs,z), 
    diff(uvbar_wp.rhs,z).replace(
        Derivative(wp(wild, g2, g3), wild),
        wpp(wild, g2, g3)
    ).doit()
)
duv_at_mu_j_wpp = duvbar_wpp.subs(z, mu[j])


wp_z1_d_lam1
wp_z0_mu_j_d
wpp_z0_mu_j_d
hz_bar_wp
uvbar_wp_d5
uvbar_wp
duvbar_wpp
duv_at_mu_j_wpp

Eq(wp(z1, g2, g3), d[2]/12 + d[3]*lambda[1]/4 + d[4]*lambda[1]**2/2)

Eq(wp(-z0 + mu[j], g2, g3), d[2]/12 + d[3]*lambda[1]/4 + d[4]*lambda[1]**2/2 - (-d[1] - 2*d[2]*lambda[1] - 3*d[3]*lambda[1]**2 - 4*d[4]*lambda[1]**3)/(4*gamma[j] - 4*lambda[1]))

Eq(\wp'(-z0 + mu[j], g2, g3), (-b[0] - b[1]*gamma[j] - b[2]*gamma[j]**2)*(d[1] + 2*d[2]*lambda[1] + 3*d[3]*lambda[1]**2 + 4*d[4]*lambda[1]**3)/(4*(gamma[j] - lambda[1])**2))

Eq(h(z), 4*wp(z1, g2, g3)*lambda[1] - 4*wp(z - z0, g2, g3)*lambda[1])

Eq(ubar(z, mu[j])*vbar(z, mu[j]), wp(z1, g2, g3) - wp(z - z0, g2, g3) + d[5]/(4*(gamma[j] - lambda[1])))

Eq(ubar(z, mu[j])*vbar(z, mu[j]), -wp(z - z0, g2, g3) + wp(-z0 + mu[j], g2, g3))

Eq(Derivative(ubar(z, mu[j])*vbar(z, mu[j]), z), -\wp'(z - z0, g2, g3))

Eq(Derivative(ubar(mu[j], mu[j])*vbar(mu[j], mu[j]), mu[j]), -\wp'(-z0 + mu[j], g2, g3))

We now solve for $\bar{u}$ and confirm they are Kronecker theta functions, thus completing the main aim:

In [42]:
# Derive log diff of u

dlog_u_bar = Eq(du_bar.lhs/ubar(z, mu[j]), (du_bar.rhs/ubar(z, mu[j])).expand())
dlog_uv_bar = Eq(
    duv_bar.lhs/(ubar(z, mu[j])*vbar(z, mu[j])), 
    (duv_bar.rhs/(ubar(z, mu[j])*vbar(z, mu[j]))).expand()
)
dlog_u_bar_duv = Eq(
    dlog_u_bar.lhs,
    dlog_u_bar.rhs + dlog_uv_bar.lhs/2 - dlog_uv_bar.rhs/2
)

H_bar_val_b_prods_uv_j = H_bar_val_b_prods.subs([hz_UV_uvbar.args]).doit().expand()
H_bar_val_b_prods_uv_j = Eq(
    (H_bar_val_b_prods_uv_j.lhs/(ubar(z, mu[j])*vbar(z, mu[j]))/2).expand(),
    (H_bar_val_b_prods_uv_j.rhs/(ubar(z, mu[j])*vbar(z, mu[j]))/2)
    .expand().collect(ubar(z, mu[j])*vbar(z, mu[j]), factor)
)
dlog_u_bar_duv_b = Eq(
    dlog_u_bar_duv.lhs,
    dlog_u_bar_duv.rhs + H_bar_val_b_prods_uv_j.lhs - H_bar_val_b_prods_uv_j.rhs
)

__const = (
    - d[5]/(8*(b[0] + b[1]*lam[1] + b[2]*lam[1]**2)) + 
    (2*b[0] + b[1]*gamma[j] + b[1]*lam[1] + 2*b[2]*gamma[j]*lam[1])/(2*(gamma[j] - lam[1]))
)
dlog_u_bar_duv_C = dlog_u_bar_duv_b.subs([
    (C, solve(C_gam_prod_sign, C)[0]),
    (__const.args[0], apart(__const.args[0], gamma[j]))
])


# Express log derivative in terms of wp

wpp_z0_mu_j_d5 = wpp_z0_mu_j_d.subs([(d5_lam1.rhs, d5_lam1.lhs)])
wpp_z0_mu_j_d5 = Eq(wpp_z0_mu_j_d5.rhs.factor()/2, wpp_z0_mu_j_d5.lhs/2)

dlog_u_bar_duv_wp = dlog_u_bar_duv_C.subs([
    duvbar_wpp.args,
    uvbar_wp.args,
    wpp_z0_mu_j_d5.args
])
dlog_u_bar_duv_wp = Eq(dlog_u_bar_duv_wp.lhs, sum(_.factor() for _ in dlog_u_bar_duv_wp.rhs.args))

def _zero(*kwrds):
    return 0

theta_j = Eq(theta[j], dlog_u_bar_duv_wp.rhs.replace(wpp,_zero))
dlog_u_bar_duv_wp = dlog_u_bar_duv_wp.subs([(theta_j.rhs, theta_j.lhs)])

# Express log derivative in terms of zeta

pw_to_zw_identity_z_mu_j = Eq(
    pw_to_zw_identity.lhs.expand(),
    pw_to_zw_identity.rhs.expand()
).subs([
    (z, z - z0),
    (x, mu[j] - z0)
])
pw_to_zw_identity_z_mu_j = Eq(
    sum(_.factor() for _ in pw_to_zw_identity_z_mu_j.lhs.args),
    pw_to_zw_identity_z_mu_j.rhs,
)
dlog_u_bar_duv_zw = dlog_u_bar_duv_wp.subs([pw_to_zw_identity_z_mu_j.args])

# Express log derivative in terms of zeta

dlog_u_bar_duv_sigma = dlog_u_bar_duv_zw.subs([
    zw_dlog_sigma_identity.subs([(x,z0)]).args,
    zw_dlog_sigma_identity.subs([(x, -mu[j] + 2*z0)]).args
])

# Where epsilon[j] is some integration constant that captures relative phase offset between modes
dlog_u_bar_duv_sigma = Eq(
    Integral(dlog_u_bar_duv_sigma.lhs,z).doit(),
    Integral(dlog_u_bar_duv_sigma.rhs,z).doit() + log(epsilon[j]/sigma(mu[j]-z0,g2,g3))
)

# Integrate to solve for ubar

ubar_sol = Eq(ubar(z, mu[j]), solve(dlog_u_bar_duv_sigma, ubar(z, mu[j]))[0])

uvbar_sigma = sigma_p_identity.subs([
    (x,z-z0),
    (y, mu[j] - z0),
    (uvbar_wp.rhs, uvbar_wp.lhs)
])

# Solve for vbar from modal power wp

vbar_sol = Eq(uvbar_sigma.lhs/ubar_sol.lhs, uvbar_sigma.rhs/ubar_sol.rhs)


# du_bar
# dlog_u_bar
# dlog_uv_bar
# dlog_u_bar_duv
# H_bar_val_b_prods_uv_j
# dlog_u_bar_duv_b


dlog_u_bar_duv_C
theta_j
dlog_u_bar_duv_wp
dlog_u_bar_duv_zw
dlog_u_bar_duv_sigma
ubar_sol
vbar_sol

Eq(Derivative(ubar(z, mu[j]), z)/ubar(z, mu[j]), b[1]/2 + b[2]*lambda[1] + Derivative(ubar(z, mu[j])*vbar(z, mu[j]), z)/(2*ubar(z, mu[j])*vbar(z, mu[j])) - d[5]/(8*b[0] + 8*b[1]*lambda[1] + 8*b[2]*lambda[1]**2) + (b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)/(gamma[j] - lambda[1]) - (b[0] + b[1]*gamma[j] + b[2]*gamma[j]**2)*d[5]/(8*(gamma[j] - lambda[1])**2*ubar(z, mu[j])*vbar(z, mu[j])))

Eq(theta[j], b[1]/2 + b[2]*lambda[1] - d[5]/(8*(b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)) + (b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)/(gamma[j] - lambda[1]))

Eq(Derivative(ubar(z, mu[j]), z)/ubar(z, mu[j]), theta[j] - \wp'(z - z0, g2, g3)/(2*(-wp(z - z0, g2, g3) + wp(-z0 + mu[j], g2, g3))) + \wp'(-z0 + mu[j], g2, g3)/(2*(-wp(z - z0, g2, g3) + wp(-z0 + mu[j], g2, g3))))

Eq(Derivative(ubar(z, mu[j]), z)/ubar(z, mu[j]), -zeta(z - z0, g2, g3) - zeta(-z0 + mu[j], g2, g3) + zeta(z - 2*z0 + mu[j], g2, g3) + theta[j])

Eq(log(ubar(z, mu[j])), -z*zeta(-z0 + mu[j], g2, g3) + z*theta[j] + log(epsilon[j]/sigma(-z0 + mu[j], g2, g3)) - log(sigma(z - z0, g2, g3)) + log(sigma(z - 2*z0 + mu[j], g2, g3)))

Eq(ubar(z, mu[j]), sigma(z - 2*z0 + mu[j], g2, g3)*exp(z*(-zeta(-z0 + mu[j], g2, g3) + theta[j]))*epsilon[j]/(sigma(z - z0, g2, g3)*sigma(-z0 + mu[j], g2, g3)))

Eq(vbar(z, mu[j]), sigma(z - mu[j], g2, g3)*exp(-z*(-zeta(-z0 + mu[j], g2, g3) + theta[j]))/(sigma(z - z0, g2, g3)*sigma(-z0 + mu[j], g2, g3)*epsilon[j]))

We can then solve for $U,V$ by reversing the gauge transform. How we manage the exponential terms is not significant, the key point is that we obtain single valued Kronecker theta functions.

In [43]:
_Cmlam_gam = (-1)**m*C/(gamma[j] - lam[1])
_Cmlam_gam_sub = (_Cmlam_gam, solve(theta_j.subs([C_gam_prod_sign.args]), _Cmlam_gam)[0])

U_sol = ubar_gauge.subs([_Cmlam_gam_sub, ubar_sol.args, ((-1)**(-m), (-1)**m)]).expand()
V_sol = vbar_gauge.subs([_Cmlam_gam_sub, vbar_sol.args, ((-1)**(-m), (-1)**m)]).expand()

U_sol
V_sol

Eq(U(z, mu[j]), sigma(z - 2*z0 + mu[j], g2, g3)*exp(-z*zeta(-z0 + mu[j], g2, g3))*exp(z*theta[j]/2)*exp(-z*b[2]*gamma[j]/2)*exp((-1)**m*z*d[5]/(32*C))*epsilon[j]/(sigma(z - z0, g2, g3)*sigma(-z0 + mu[j], g2, g3)))

Eq(V(z, mu[j]), sigma(z - mu[j], g2, g3)*exp(z*zeta(-z0 + mu[j], g2, g3))*exp(-z*theta[j]/2)*exp(z*b[2]*gamma[j]/2)*exp(-(-1)**m*z*d[5]/(32*C))/(sigma(z - z0, g2, g3)*sigma(-z0 + mu[j], g2, g3)*epsilon[j]))

## The parameterless system solutions

In this section we will solve the parameterless system. We recall that:

In [44]:
H_tilde_uv_j_val_c
H_tilde_uv_j

ubar_to_utilde
vbar_to_vtilde
z_t_d5
du_tilde
dv_tilde

Eq(Htilde, -8*(b[1] + 2*b[2]*lambda[1])*(b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)/d[5] - 32*(b[0] + b[1]*lambda[1] + b[2]*lambda[1]**2)**2*(b[0]*b[2] - b[1]**2 - 3*b[1]*b[2]*lambda[1] - 3*b[2]**2*lambda[1]**2 + 2*c[2] + 12*lambda[1]**2)/d[5]**2 - 4)

Eq(Htilde, -Product(utilde(t, mu[l]), (l, 1, 4)) - Product(vtilde(t, mu[l]), (l, 1, 4)) - Sum(utilde(t, mu[j])*vtilde(t, mu[j]), (j, 1, 4)) + Sum(utilde(t, mu[j])**2*vtilde(t, mu[j])**2, (j, 1, 4))/2)

Eq(ubar(z, mu[j]), utilde((-1)**m*z*d[5]/(16*C), mu[j])*exp(-I*pi*m/4)*d[5]/(8*C))

Eq(vbar(z, mu[j]), vtilde((-1)**m*z*d[5]/(16*C), mu[j])*exp(I*pi*m/4)*d[5]/(8*C))

Eq(z, 16*C*t/((-1)**m*d[5]))

Eq(Derivative(utilde(t, mu[j]), t), utilde(t, mu[j])**2*vtilde(t, mu[j]) - utilde(t, mu[j]) - vtilde(t, mu[1])*vtilde(t, mu[2])*vtilde(t, mu[3])*vtilde(t, mu[4])/vtilde(t, mu[j]))

Eq(Derivative(vtilde(t, mu[j]), t), utilde(t, mu[1])*utilde(t, mu[2])*utilde(t, mu[3])*utilde(t, mu[4])/utilde(t, mu[j]) - utilde(t, mu[j])*vtilde(t, mu[j])**2 + vtilde(t, mu[j]))

We further introduce the function $\tilde{h}(t)$:

In [45]:
htilde_z = Eq(htilde(z), 64*C**2*h(16*(-1)**m*C*z/d[5])/d[5]**2/lam[1])
uvtilde_hz = uvbar_hz.subs([
    (z_t_d5.args),
    ubar_to_utilde.subs([z_t_d5.args]).args,
    vbar_to_vtilde.subs([z_t_d5.args]).args,
    ((-1)**(-m), (-1)**(m))
])

uvtilde_hz = Eq(
    uvtilde_hz.lhs*64*C**2/d[5]**2, 
    (uvtilde_hz.rhs*64*C**2/d[5]**2).expand().collect(d[5], factor)
    .subs([
        (htilde_z.rhs.subs(z,t)/64, htilde_z.lhs.subs(z,t)/64)
    ])
)

htilde_z
uvtilde_hz

Eq(htilde(z), 64*C**2*h(16*(-1)**m*C*z/d[5])/(d[5]**2*lambda[1]))

Eq(utilde(t, mu[j])*vtilde(t, mu[j]), 16*C**2/((gamma[j] - lambda[1])*d[5]) + htilde(t)/4)

In [46]:
# Convert sum terms to h

uvtilde_hz_sum = Eq(
    Sum(utilde(t, mu[j])*vtilde(t, mu[j]),(j,1,4)),
    htilde(t) + 16*C**2/d[5]*Sum(1/(gamma[j] - lam[1]),(j,1,4))
)

uvtilde_hz_sum_check = (Sum(uvtilde_hz.rhs, (j,1,4)) - uvtilde_hz_sum.rhs).doit().simplify() == 0
if not uvtilde_hz_sum_check:
    raise ValueError('uvtilde_hz_sum_check failed')

uvtilde_hz_sqrd = Eq(uvtilde_hz.lhs**2, (uvtilde_hz.rhs**2).expand().collect(htilde(t), factor))
uvtilde_hz_sqrd_sum = Eq(
    Sum(utilde(t, mu[j])**2*vtilde(t, mu[j])**2,(j,1,4)),
    256*C**4/d[5]**2*Sum(1/(gamma[j] - lam[1])**2,(j,1,4)) + 
    8*C**2*htilde(t)/d[5]*Sum(1/(gamma[j] - lam[1]),(j,1,4)) + 
    htilde(t)**2/4
)

uvtilde_hz_sqrd_sum_check = (Sum(uvtilde_hz_sqrd.rhs, (j,1,4)) - uvtilde_hz_sqrd_sum.rhs).doit().simplify() == 0
if not uvtilde_hz_sqrd_sum_check:
    raise ValueError('uvtilde_hz_sqrd_sum_check failed')

uvtilde_hz_sum_b = uvtilde_hz_sum.subs(gam_sum_d5_1_subs)
uvtilde_hz_sqrd_sum_b = uvtilde_hz_sqrd_sum.subs(gam_sum_d5_1_subs)
uvtilde_hz_sum_C = uvtilde_hz_sum.subs(gam_sum_d5_1_C_subs)
uvtilde_hz_sqrd_sum_C = uvtilde_hz_sqrd_sum.subs(gam_sum_d5_1_C_subs)

# Sub in sums in terms of h into H

H_tilde_uv_j_h = H_tilde_uv_j.subs([
    uvtilde_hz_sum_C.args,
    uvtilde_hz_sqrd_sum_C.args
])
H_tilde_uv_j_h = Eq(
    H_tilde_uv_j_h.lhs, 
    H_tilde_uv_j_h.rhs.expand().collect([htilde(t), d[5]], factor)
)



# uvtilde_hz_sum
# uvtilde_hz_sqrd_sum
uvtilde_hz_sum_C
uvtilde_hz_sqrd_sum_C
H_tilde_uv_j_h

Eq(Sum(utilde(t, mu[j])*vtilde(t, mu[j]), (j, 1, 4)), 4*(-2*(-1)**m*C*(2*b[1] + 4*b[2]*lambda[1]) + d[5])/d[5] + htilde(t))

Eq(Sum(utilde(t, mu[j])**2*vtilde(t, mu[j])**2, (j, 1, 4)), 256*C**4*(8*(-1)**m*b[2]/C - (-1)**m*(b[1] + 2*b[2]*lambda[1])*d[5]/(4*C**3) + (-4*b[0]*b[2] + b[1]**2 - 2*c[2] - 12*lambda[1]**2)/C**2)/d[5]**2 + 2*(-2*(-1)**m*C*(2*b[1] + 4*b[2]*lambda[1]) + d[5])*htilde(t)/d[5] + htilde(t)**2/4)

Eq(Htilde, -4*(-1)**m*C*(b[1] + 2*b[2]*lambda[1])*htilde(t)/d[5] - 16*(-1)**m*C*(b[1] + 2*b[2]*lambda[1])/d[5] + 128*C**2*(8*(-1)**m*C*b[2] - 4*b[0]*b[2] + b[1]**2 - 2*c[2] - 12*lambda[1]**2)/d[5]**2 + htilde(t)**2/8 - Product(utilde(t, mu[l]), (l, 1, 4)) - Product(vtilde(t, mu[l]), (l, 1, 4)) - 4)

In [47]:
const_term_h_tilde = H_tilde_uv_j_h.rhs.replace(l,j).subs([
    (htilde(t),0), (utilde(t, mu[j]),0), (vtilde(t, mu[j]),0)
]).doit()

H_tilde_val_min_h_const = Eq(
    (H_tilde_uv_j_val_c.lhs - const_term_h_tilde),
    (H_tilde_uv_j_val_c.rhs - const_term_h_tilde).expand().collect(C, factor).expand().subs([
        ((-1)**(2*m),1),
        (C, solve(C_gam_prod_sign, C)[0]),
        ((-1)**(-2*m),1),
    ])
    .expand().simplify().factor()
    .subs([C_gam_prod_sign.args, ((-1)**(3*m),(-1)**m),])
)

H_tilde_uv_j_h_b = Eq(
    (H_tilde_uv_j_h.lhs - const_term_h_tilde).subs([H_tilde_val_min_h_const.args]),
    (H_tilde_uv_j_h.rhs - const_term_h_tilde).subs([((-1)**(2*m),1)])
)

H_tilde_val_min_h_const
H_tilde_uv_j_h_b


Eq(16*(-1)**m*C*(b[1] + 2*b[2]*lambda[1])/d[5] - 128*C**2*(8*(-1)**m*C*b[2] - 4*b[0]*b[2] + b[1]**2 - 2*c[2] - 12*lambda[1]**2)/d[5]**2 + Htilde + 4, -256*(-1)**m*C**3*b[2]/d[5]**2)

Eq(-256*(-1)**m*C**3*b[2]/d[5]**2, -4*(-1)**m*C*(b[1] + 2*b[2]*lambda[1])*htilde(t)/d[5] + htilde(t)**2/8 - Product(utilde(t, mu[l]), (l, 1, 4)) - Product(vtilde(t, mu[l]), (l, 1, 4)))

In [48]:
# Build the product term

uv_tilde_prod_h_poly = (
    uv_bar_prod_h_poly
    .subs([
        z_t_d5.args,
        ((-1)**(-m), (-1)**(m)),
        (
            h(z.subs(*z_t_d5.args)).subs([((-1)**(-m), (-1)**(m))]), 
            solve(htilde_z.subs([(z,t), ((-1)**(-m), (-1)**(m))]), h(z.subs(*z_t_d5.args)))[0]
        )
    ])
    .replace(*ubar_to_utilde.subs([z_t_d5.args, ((-1)**(-m), (-1)**(m))]).args)
    .replace(*vbar_to_vtilde.subs([z_t_d5.args, ((-1)**(-m), (-1)**(m))]).args)
)
uv_tilde_prod_h_poly = Eq(
    uv_tilde_prod_h_poly.lhs.doit()*(
        Product(utilde(t, mu[j])*vtilde(t, mu[j]), (j,1,4))/
        Product(utilde(t, mu[j])*vtilde(t, mu[j]), (j,1,4)).doit()
    ),
    uv_tilde_prod_h_poly.rhs
)
uv_tilde_prod_h_poly = Eq(
    Product(utilde(t, mu[j])*vtilde(t, mu[j]), (j,1,4)),
    solve(uv_tilde_prod_h_poly, Product(utilde(t, mu[j])*vtilde(t, mu[j]), (j,1,4)))[0]
    .expand().collect(htilde(t), factor)
)

uv_tilde_prod_h_poly

Eq(Product(utilde(t, mu[j])*vtilde(t, mu[j]), (j, 1, 4)), 65536*C**6/d[5]**4 - 4096*C**4*htilde(t)*lambda[1]/d[5]**3 + 16*C**2*(c[2] + 6*lambda[1]**2)*htilde(t)**2/d[5]**2 - (c[1] + 2*c[2]*lambda[1] + 4*lambda[1]**3)*htilde(t)**3/(4*d[5]) + htilde(t)**4/256)

In [49]:
H_tilde_val_b_prods = Eq(
    H_tilde_uv_j_h_b.lhs - H_tilde_uv_j_h_b.lhs - H_tilde_uv_j_h_b.rhs.subs(htilde(t),0),
    H_tilde_uv_j_h_b.rhs - H_tilde_uv_j_h_b.lhs - H_tilde_uv_j_h_b.rhs.subs(htilde(t),0)
)

duv_tilde = Eq(
    Derivative(utilde(t, mu[j])*vtilde(t, mu[j]),t),
    Derivative(utilde(t, mu[j])*vtilde(t, mu[j]),t).doit().subs([
        du_tilde.args,
        dv_tilde.args
    ])
    .expand().simplify()
)

dh_uv_tilde_sqrd = Eq(
    16*(duv_tilde.lhs**2).subs([uvtilde_hz.args]).doit(), 
    (
        16*(
            (duv_tilde.rhs**2).expand()
            - ((H_tilde_val_b_prods.lhs)**2).doit().expand() + H_tilde_val_b_prods.rhs**2
        ).subs([
            uv_tilde_prod_h_poly.doit().args
        ])
        .expand().subs([((-1)**(2*m),1)]).collect(htilde(t), factor)
        .subs([
            (C_gam_prod_sign.rhs/2, C_gam_prod_sign.lhs/2)
        ])
    ).expand().collect(htilde(t), factor)
)


dh_uv_tilde_sqrd = Eq(
    dh_uv_tilde_sqrd.lhs, 
    sum(_.collect(lam[1]) for _ in dh_uv_tilde_sqrd.rhs.args)
)

dh_uv_tilde_sqrd_to_d = Eq(
    dh_uv_tilde_sqrd.lhs, 
    dh_uv_tilde_sqrd.rhs.subs(to_d_subs).collect(h(z),expand).subs(to_d_subs).collect(htilde(t),factor)
    .subs([(d5_lam1.rhs, d5_lam1.lhs)])
)

duv_tilde
dh_uv_tilde_sqrd_to_d

Eq(Derivative(utilde(t, mu[j])*vtilde(t, mu[j]), t), utilde(t, mu[1])*utilde(t, mu[2])*utilde(t, mu[3])*utilde(t, mu[4]) - vtilde(t, mu[1])*vtilde(t, mu[2])*vtilde(t, mu[3])*vtilde(t, mu[4]))

Eq(Derivative(htilde(t), t)**2, 1048576*C**6*d[4]/d[5]**4 - 16384*C**4*(d[3] + 4*d[4]*lambda[1])*htilde(t)/d[5]**3 + 256*C**2*(d[2] + 3*d[3]*lambda[1] + 6*d[4]*lambda[1]**2)*htilde(t)**2/d[5]**2 - 4*htilde(t)**3)

In [50]:
htilde_to_w_eps = Eq(htilde(t), -wtilde(t) + epsilon)
dw_tilde = dh_uv_tilde_sqrd_to_d.subs([htilde_to_w_eps.args]).doit()
dw_tilde = Eq(dw_tilde.lhs, dw_tilde.rhs.expand().collect(wtilde(t), factor))
eps_sol_tilde = Eq(epsilon, solve(dw_tilde.rhs.coeff(wtilde(t),2),epsilon)[0])
T_scale = Eq(T, 16*C/d[5])
dw_tilde = Eq(
    dw_tilde.lhs, 
    dw_tilde.rhs.subs([eps_sol_tilde.args])
    .expand().collect(wtilde(t), simplify)
    .subs(lam[1]**4, solve(lam1_root, lam[1]**4)[0])
    .expand().collect(wtilde(t), simplify)
    .subs([(C, solve(T_scale,C)[0]), d5_lam1.args])
    .expand().collect(wtilde(t), simplify)
    .subs(lam[1]**4, solve(lam1_root, lam[1]**4)[0])
    .expand().collect(wtilde(t), simplify)
)
dw_tilde_g2_g3 = dw_tilde.subs([(g2_dj.rhs, g2_dj.lhs), (g3_dj.rhs, g3_dj.lhs)])
h_to_w_tilde = htilde_to_w_eps.subs([eps_sol_tilde.args])
gtilde2_eq = Eq(gtilde2, T**4*g2)
gtilde3_eq = Eq(gtilde3, T**6*g2)
wtidle_wp = Eq(wtilde(t), wp(t-t0, gtilde2, gtilde3))

T_scale
dw_tilde
dw_tilde_g2_g3
h_to_w_tilde
g2_dj
g3_dj
gtilde2_eq
gtilde3_eq
wtidle_wp

Eq(T, 16*C/d[5])

Eq(Derivative(wtilde(t), t)**2, T**6*(-d[0]*d[2]*d[4]/6 + d[0]*d[3]**2/16 + d[1]**2*d[4]/16 - d[1]*d[2]*d[3]/48 + d[2]**3/216) + T**4*(-d[0]*d[4] + d[1]*d[3]/4 - d[2]**2/12)*wtilde(t) + 4*wtilde(t)**3)

Eq(Derivative(wtilde(t), t)**2, -T**6*g3 - T**4*g2*wtilde(t) + 4*wtilde(t)**3)

Eq(htilde(t), 64*C**2*(d[2] + 3*d[3]*lambda[1] + 6*d[4]*lambda[1]**2)/(3*d[5]**2) - wtilde(t))

Eq(g2, d[0]*d[4] - d[1]*d[3]/4 + d[2]**2/12)

Eq(g3, d[0]*d[2]*d[4]/6 - d[0]*d[3]**2/16 - d[1]**2*d[4]/16 + d[1]*d[2]*d[3]/48 - d[2]**3/216)

Eq(gtilde2, T**4*g2)

Eq(gtilde3, T**6*g2)

Eq(wtilde(t), wp(t - t0, gtilde2, gtilde3))

### Invariants in terms of initial conditions only

We note that the cross ratio is preserved between cooridnate transformations. Furthermore, we note that the scaling factor which relates the parameterless system to the original system coordinates can be expressed in terms of the initial conditions. In hindsight, this was obvious from the definition of the functions in the parameterless system, nonetheless it may be important.

Because the parameterless system has no parameters, we shoudl be able to also express $\tilde{g_2},\tilde{g_3}$ exclusively in terms of initial conditions. It would be very interesting if we could then also express $g_2, g_3$ in terms of initial conditions in those coordinates, which is not something we have yet shown. It may enable separation of concerns in terms of optimisation of nonlinear processes if one could optimise the geometry of the initial conditions (transcendental, elliptic function theory), and the scale factor in terms of system parameters separately (algebraic).

In [51]:
# j != k != l != m

tilde_pow_cons = Eq(uvtilde_hz.lhs - uvtilde_hz.lhs.subs(j,k), uvtilde_hz.rhs - uvtilde_hz.rhs.subs(j,k))
tilde_pow_cons_b = Eq(
    tilde_pow_cons.lhs*(gamma[j] - lam[1])*(gamma[k] - lam[1]),
    (tilde_pow_cons.rhs*(gamma[j] - lam[1])*(gamma[k] - lam[1])).factor()
)

gamma_jklm = Eq(
    (gamma[j] - lam[1])*(gamma[k] - lam[1])*(gamma[l] - lam[1])*(gamma[m] - lam[1]),
    Product(gamma[n] - lam[1], (n,1,4))
)
gamm_pow_diff_uv = Eq(gamma[j] - gamma[k], u(z, mu[j])*v(z, mu[j]) - u(z, mu[k])*v(z, mu[k]))

uv_tilde_gamma_jk_C_d5 = Eq(
    (
        tilde_pow_cons_b.lhs*
        tilde_pow_cons_b.lhs.subs([(j,l),(k,m)])
    ).subs([
        gamma_jklm.doit().args,
        gamma_j_prod_b_poly.subs([(C_gam_prod_sqrd.rhs, C_gam_prod_sqrd.lhs)]).doit().args
    ])/C**2
    ,
    tilde_pow_cons_b.rhs*
    tilde_pow_cons_b.rhs.subs([(j,l),(k,m)])/C**2
)

gam_jk_lm_diff_prod = (gamma[j] - gamma[k])*(gamma[l] - gamma[m])
uv_tilde_uv_ratio_C_d5 = Eq(
    uv_tilde_gamma_jk_C_d5.lhs/gam_jk_lm_diff_prod.subs([
        gamm_pow_diff_uv.args,
        gamm_pow_diff_uv.subs([(j,l),(k,m)]).args
    ]),
    uv_tilde_gamma_jk_C_d5.rhs/gam_jk_lm_diff_prod
)

uv_tilde_uv_ratio_T_d5_0 = uv_tilde_uv_ratio_C_d5.subs([(z,0),(t,0), (C, solve(T_scale, C)[0])])

# Cross ratio is preserved between coordinate systems
perm_jklm = [(k,x),(l,y),(j,k),(m,l),(x,m),(y,j)]
uv_tilde_uv_cross_ratio = Eq(
    uv_tilde_gamma_jk_C_d5.lhs/uv_tilde_gamma_jk_C_d5.lhs.subs(perm_jklm),
    (
        uv_tilde_gamma_jk_C_d5.rhs.subs([gamm_pow_diff_uv.args])/
        uv_tilde_gamma_jk_C_d5.rhs.subs(perm_jklm).subs([
            gamm_pow_diff_uv.args,
            gamm_pow_diff_uv.subs([(k,l)]).args,
            gamm_pow_diff_uv.subs([(k,m),(j,k)]).args,
        ])
    ).subs([
        gamm_pow_diff_uv.subs([(j,l),(k, m)]).args
    ])
)


tilde_pow_cons
tilde_pow_cons_b

# gamm_pow_diff_uv

uv_tilde_gamma_jk_C_d5
uv_tilde_uv_ratio_C_d5
uv_tilde_uv_ratio_T_d5_0
uv_tilde_uv_cross_ratio

Eq(utilde(t, mu[j])*vtilde(t, mu[j]) - utilde(t, mu[k])*vtilde(t, mu[k]), -16*C**2/((gamma[k] - lambda[1])*d[5]) + 16*C**2/((gamma[j] - lambda[1])*d[5]))

Eq((utilde(t, mu[j])*vtilde(t, mu[j]) - utilde(t, mu[k])*vtilde(t, mu[k]))*(gamma[j] - lambda[1])*(gamma[k] - lambda[1]), -16*C**2*(gamma[j] - gamma[k])/d[5])

Eq((utilde(t, mu[j])*vtilde(t, mu[j]) - utilde(t, mu[k])*vtilde(t, mu[k]))*(utilde(t, mu[l])*vtilde(t, mu[l]) - utilde(t, mu[m])*vtilde(t, mu[m])), 256*C**2*(gamma[j] - gamma[k])*(gamma[l] - gamma[m])/d[5]**2)

Eq((utilde(t, mu[j])*vtilde(t, mu[j]) - utilde(t, mu[k])*vtilde(t, mu[k]))*(utilde(t, mu[l])*vtilde(t, mu[l]) - utilde(t, mu[m])*vtilde(t, mu[m]))/((u(z, mu[j])*v(z, mu[j]) - u(z, mu[k])*v(z, mu[k]))*(u(z, mu[l])*v(z, mu[l]) - u(z, mu[m])*v(z, mu[m]))), 256*C**2/d[5]**2)

Eq((utilde(0, mu[j])*vtilde(0, mu[j]) - utilde(0, mu[k])*vtilde(0, mu[k]))*(utilde(0, mu[l])*vtilde(0, mu[l]) - utilde(0, mu[m])*vtilde(0, mu[m]))/((u(0, mu[j])*v(0, mu[j]) - u(0, mu[k])*v(0, mu[k]))*(u(0, mu[l])*v(0, mu[l]) - u(0, mu[m])*v(0, mu[m]))), T**2)

Eq((utilde(t, mu[j])*vtilde(t, mu[j]) - utilde(t, mu[k])*vtilde(t, mu[k]))*(utilde(t, mu[l])*vtilde(t, mu[l]) - utilde(t, mu[m])*vtilde(t, mu[m]))/((utilde(t, mu[j])*vtilde(t, mu[j]) - utilde(t, mu[l])*vtilde(t, mu[l]))*(utilde(t, mu[k])*vtilde(t, mu[k]) - utilde(t, mu[m])*vtilde(t, mu[m]))), (u(z, mu[j])*v(z, mu[j]) - u(z, mu[k])*v(z, mu[k]))*(u(z, mu[l])*v(z, mu[l]) - u(z, mu[m])*v(z, mu[m]))/((u(z, mu[j])*v(z, mu[j]) - u(z, mu[l])*v(z, mu[l]))*(u(z, mu[k])*v(z, mu[k]) - u(z, mu[m])*v(z, mu[m]))))

To proceed, we introduce $s(z)$ and related it to modal powers through initial conditions:

In [52]:
uv_tilde_s = Eq(utilde(t, mu[j])*vtilde(t, mu[j]), upsilon[j] - s(t))
upsilon_sum = Eq(Sum(upsilon[j],(j,1,4)),0)
s_sum_uv_tilde = Eq(Sum(utilde(t, mu[j])*vtilde(t, mu[j]),(j,1,4))/4,  - s(t))
s_sum_uv_tilde_0 = Eq(-s_sum_uv_tilde.rhs.subs(t,0), -s_sum_uv_tilde.lhs.subs(t,0)).replace(j,l)
upsilon_0 = Eq(upsilon[j], solve(uv_tilde_s, upsilon[j])[0].subs(t,0).subs([s_sum_uv_tilde_0.args]))
uv_tilde_s_uv0 = uv_tilde_s.subs([upsilon_0.args])

uv_tilde_s
upsilon_sum
s_sum_uv_tilde
s_sum_uv_tilde_0
upsilon_0
uv_tilde_s_uv0

Eq(utilde(t, mu[j])*vtilde(t, mu[j]), -s(t) + upsilon[j])

Eq(Sum(upsilon[j], (j, 1, 4)), 0)

Eq(Sum(utilde(t, mu[j])*vtilde(t, mu[j]), (j, 1, 4))/4, -s(t))

Eq(s(0), -Sum(utilde(0, mu[l])*vtilde(0, mu[l]), (l, 1, 4))/4)

Eq(upsilon[j], utilde(0, mu[j])*vtilde(0, mu[j]) - Sum(utilde(0, mu[l])*vtilde(0, mu[l]), (l, 1, 4))/4)

Eq(utilde(t, mu[j])*vtilde(t, mu[j]), -s(t) + utilde(0, mu[j])*vtilde(0, mu[j]) - Sum(utilde(0, mu[l])*vtilde(0, mu[l]), (l, 1, 4))/4)

We immediately see that the constant $\Psi_4$ that we derive from the Hamiltonian contains terms both linear and quadratic in modal powers. This is annoying because it means this constant does not scale in proportion to the scaling of the fields, rather it is ... nonlinear. This is reminding us that in a physical setting, the system genuninely does depend on the ratio of dispersive to nonlinear effects.

In [53]:
# Get the value of the Hamiltonian in terms of init conditions

H_tilde_uv_j_0 = H_tilde_uv_j.subs(t,0)

Htilde_st = Eq(
    Htilde,
    -Product(utilde(t, mu[l]), (l, 1, 4)) - Product(vtilde(t, mu[l]), (l, 1, 4)) 
    + 2*s(t)**2 - Sum(utilde(0, mu[l])*vtilde(0, mu[l]), (l, 1, 4))**2/8 + 
    Sum(utilde(0, mu[l])**2*vtilde(0, mu[l])**2, (l, 1, 4))/2 +
    4*s(t)
)

Htilde_st_check = (
    H_tilde_uv_j
    .replace(*uv_tilde_s_uv0.args)
    .replace(uv_tilde_s_uv0.lhs**2, (uv_tilde_s_uv0.rhs**2).expand()).rhs
    - Htilde_st.rhs
).doit().simplify().expand().collect(s(t)**2) == 0
if not Htilde_st_check:
    raise ValueError('Htilde_st_check failed')

# Express product terms in s and the constant
uv_tilde_prods_mix = Product(utilde(t, mu[j]), (j, 1, 4)) + Product(vtilde(t, mu[j]), (j, 1, 4))
uv_tilde_prods_s = Eq(
    Htilde_st.lhs - H_tilde_uv_j_0.lhs + uv_tilde_prods_mix,
    (Htilde_st.rhs - H_tilde_uv_j_0.rhs).replace(l,j) + uv_tilde_prods_mix
)
uv_tilde_const_no_s = Eq(Psi[4], uv_tilde_prods_s.rhs.subs(s(t),0))
uv_tilde_prods_s = uv_tilde_prods_s.subs([(uv_tilde_const_no_s.rhs, uv_tilde_const_no_s.lhs)])


H_tilde_uv_j_0
H_tilde_uv_j
Htilde_st

uv_tilde_const_no_s
uv_tilde_prods_s

Eq(Htilde, -Product(utilde(0, mu[l]), (l, 1, 4)) - Product(vtilde(0, mu[l]), (l, 1, 4)) - Sum(utilde(0, mu[j])*vtilde(0, mu[j]), (j, 1, 4)) + Sum(utilde(0, mu[j])**2*vtilde(0, mu[j])**2, (j, 1, 4))/2)

Eq(Htilde, -Product(utilde(t, mu[l]), (l, 1, 4)) - Product(vtilde(t, mu[l]), (l, 1, 4)) - Sum(utilde(t, mu[j])*vtilde(t, mu[j]), (j, 1, 4)) + Sum(utilde(t, mu[j])**2*vtilde(t, mu[j])**2, (j, 1, 4))/2)

Eq(Htilde, 2*s(t)**2 + 4*s(t) - Product(utilde(t, mu[l]), (l, 1, 4)) - Product(vtilde(t, mu[l]), (l, 1, 4)) - Sum(utilde(0, mu[l])*vtilde(0, mu[l]), (l, 1, 4))**2/8 + Sum(utilde(0, mu[l])**2*vtilde(0, mu[l])**2, (l, 1, 4))/2)

Eq(Psi[4], Product(utilde(0, mu[j]), (j, 1, 4)) + Product(vtilde(0, mu[j]), (j, 1, 4)) - Sum(utilde(0, mu[j])*vtilde(0, mu[j]), (j, 1, 4))**2/8 + Sum(utilde(0, mu[j])*vtilde(0, mu[j]), (j, 1, 4)))

Eq(Product(utilde(t, mu[j]), (j, 1, 4)) + Product(vtilde(t, mu[j]), (j, 1, 4)), 2*s(t)**2 + 4*s(t) + Psi[4])

In [54]:
# Get the mixed product in terms of s and other constant defined by init conditions only

uv_prods_as_s = Eq(
    Product(utilde(t, mu[l])*vtilde(t, mu[l]), (l, 1, 4)),
    Product((utilde(t, mu[l])*vtilde(t, mu[l])).subs([uv_tilde_s.subs(j,l).args]), (l, 1, 4))
)

uv_prods_as_s_b = Eq(
    uv_prods_as_s.lhs,
    uv_prods_as_s.rhs.doit().expand().collect(s(t), factor)
    .subs([upsilon_sum.doit().args])
)

Psi_coeffs = [
    Eq(Psi[0], Product(upsilon[l],(l,1,4))),
    Eq(Psi[1], -Sum(Product(upsilon[k],(k,1,l-1))*Product(upsilon[k],(k,l+1,4)),(l,1,4))),
    Eq(Psi[2], -Sum(upsilon[l]**2, (l,1,4))/2),
    uv_tilde_const_no_s
]
uv_prods_as_s_c = Eq(
    uv_prods_as_s.lhs,
    s(t)**4 + Psi[2]*s(t)**2  + Psi[1]*s(t) + Psi[0]
)

uv_prods_as_s_c_check = (
    uv_prods_as_s_b.rhs - uv_prods_as_s_c.rhs.subs([_.args for _ in Psi_coeffs])
).expand().doit().subs(upsilon[4], -upsilon[1]-upsilon[2]-upsilon[3]).simplify() == 0
if not uv_prods_as_s_c_check:
    raise ValueError('uv_prods_as_s_c_check failed')

uv_prods_as_s
uv_prods_as_s_b
uv_prods_as_s_c

Eq(Product(utilde(t, mu[l])*vtilde(t, mu[l]), (l, 1, 4)), Product(-s(t) + upsilon[l], (l, 1, 4)))

Eq(Product(utilde(t, mu[l])*vtilde(t, mu[l]), (l, 1, 4)), (-upsilon[1]*upsilon[2]*upsilon[3] - upsilon[1]*upsilon[2]*upsilon[4] - upsilon[1]*upsilon[3]*upsilon[4] - upsilon[2]*upsilon[3]*upsilon[4])*s(t) + (upsilon[1]*upsilon[2] + upsilon[1]*upsilon[3] + upsilon[1]*upsilon[4] + upsilon[2]*upsilon[3] + upsilon[2]*upsilon[4] + upsilon[3]*upsilon[4])*s(t)**2 + s(t)**4 + upsilon[1]*upsilon[2]*upsilon[3]*upsilon[4])

Eq(Product(utilde(t, mu[l])*vtilde(t, mu[l]), (l, 1, 4)), s(t)**4 + s(t)**2*Psi[2] + s(t)*Psi[1] + Psi[0])

We can then construct the Weierstrass differential equation and get the expressions for $\tilde{g_2}, \tilde{g_3}$ in terms of initial conditions $\Psi_k$:

In [55]:
dst_sqrd = Eq(
    (duv_tilde.lhs**2).subs([uv_tilde_s_uv0.args]).doit(),
    (
        (duv_tilde.rhs**2).expand()
        - ( (uv_tilde_prods_s.lhs**2).doit().expand() - (uv_tilde_prods_s.rhs**2).doit().expand())
        + (4*uv_prods_as_s_c.lhs.doit() - 4*uv_prods_as_s_c.rhs)
    ).expand().collect(s(t), factor) 
)

wst = (Eq(s(t), ws(t)/4 + epsilon))
dwst_sqrd = dst_sqrd.subs([wst.args])
dwst_sqrd = Eq(16*dwst_sqrd.lhs.doit(), (16*dwst_sqrd.rhs).expand().collect(ws(t), factor))
dwst_sqrd_eps_sol = Eq(epsilon, solve(dwst_sqrd.rhs.coeff(ws(t),2), epsilon)[0])
dwst_sqrd = Eq(dwst_sqrd.lhs, dwst_sqrd.rhs.subs([dwst_sqrd_eps_sol.args]).expand().collect(ws(t), factor))
gtilde2_wst = Eq(gtilde2, -dwst_sqrd.rhs.coeff(ws(t),1).factor())
gtilde3_wst = Eq(gtilde3, -dwst_sqrd.rhs.coeff(ws(t),0).expand())
dwst_sqrd = dwst_sqrd.subs([
    (gtilde2_wst.rhs, gtilde2_wst.lhs),
    (gtilde3_wst.rhs, gtilde3_wst.lhs)
])
wst = wst.subs([dwst_sqrd_eps_sol.args])


duv_tilde
dst_sqrd
wst
gtilde2_wst
gtilde3_wst
dwst_sqrd

Eq(Derivative(utilde(t, mu[j])*vtilde(t, mu[j]), t), utilde(t, mu[1])*utilde(t, mu[2])*utilde(t, mu[3])*utilde(t, mu[4]) - vtilde(t, mu[1])*vtilde(t, mu[2])*vtilde(t, mu[3])*vtilde(t, mu[4]))

Eq(Derivative(s(t), t)**2, -4*(Psi[1] - 2*Psi[4])*s(t) - 4*(Psi[2] - Psi[4] - 4)*s(t)**2 + 16*s(t)**3 - 4*Psi[0] + Psi[4]**2)

Eq(s(t), ws(t)/4 + Psi[2]/12 - Psi[4]/12 - 1/3)

Eq(gtilde2, 4*(12*Psi[1] + Psi[2]**2 - 2*Psi[2]*Psi[4] - 8*Psi[2] + Psi[4]**2 - 16*Psi[4] + 16)/3)

Eq(gtilde3, 64*Psi[0] + 16*Psi[1]*Psi[2]/3 - 16*Psi[1]*Psi[4]/3 - 64*Psi[1]/3 + 8*Psi[2]**3/27 - 8*Psi[2]**2*Psi[4]/9 - 32*Psi[2]**2/9 + 8*Psi[2]*Psi[4]**2/9 - 32*Psi[2]*Psi[4]/9 + 128*Psi[2]/9 - 8*Psi[4]**3/27 - 80*Psi[4]**2/9 + 256*Psi[4]/9 - 512/27)

Eq(Derivative(ws(t), t)**2, -gtilde2*ws(t) - gtilde3 + 4*ws(t)**3)

## General transformations of modes

Let us explore aribitrary transformations of the following form where $\hat{u},\hat{v}$ are some starting modes with intermodal power conservation and $U,V$ the new modes with some normalising function $h$ and some weights $a_j$. There is no connection otherwise to previously defined variables of the same names.

In [74]:
_uhat = Eq(uhat(z, mu[j]), sqrt(a[j])*U(z, mu[j])/sqrt(h(z)))
_vhat = Eq(vhat(z, mu[j]), sqrt(a[j])*V(z, mu[j])/sqrt(h(z)))
_uvhat = Eq(_uhat.lhs*_vhat.lhs, _uhat.rhs*_vhat.rhs)
_uv_pow_cons = Eq(
    uhat(z, mu[j])*vhat(z, mu[j]) - uhat(z, mu[k])*vhat(z, mu[k]),
    gamma[j] - gamma[k]
)

_uhat
_vhat
_uvhat
_uv_pow_cons

Eq(uhat(z, mu[j]), U(z, mu[j])*sqrt(a[j])/sqrt(h(z)))

Eq(vhat(z, mu[j]), V(z, mu[j])*sqrt(a[j])/sqrt(h(z)))

Eq(uhat(z, mu[j])*vhat(z, mu[j]), U(z, mu[j])*V(z, mu[j])*a[j]/h(z))

Eq(uhat(z, mu[j])*vhat(z, mu[j]) - uhat(z, mu[k])*vhat(z, mu[k]), gamma[j] - gamma[k])

In [109]:
Eq(
    (_uvhat.lhs*h(z)).subs(uhat(z, mu[j])*vhat(z, mu[j]), gamma[j] - rho(z)).expand(), 
    _uvhat.rhs*h(z)
)

Eq(-h(z)*rho(z) + h(z)*gamma[j], U(z, mu[j])*V(z, mu[j])*a[j])

In [83]:
Eq(
    Derivative(_uvhat.lhs,z)*h(z) + _uvhat.lhs*diff(h(z),z),
    a[j]*Derivative(_uvhat.rhs*h(z)/a[j],z)
)

Eq(h(z)*Derivative(uhat(z, mu[j])*vhat(z, mu[j]), z) + uhat(z, mu[j])*vhat(z, mu[j])*Derivative(h(z), z), Derivative(U(z, mu[j])*V(z, mu[j]), z)*a[j])

In [80]:
Eq(
    1/(
        (_uvhat.lhs - _uvhat.lhs.subs(j,k)).subs([_uv_pow_cons.args])
        /_uvhat.lhs.subs(j,k)
    )*(gamma[j] - gamma[k]), 
    (gamma[j] - gamma[k])/(_uvhat.rhs/_uvhat.rhs.subs(j,k) - 1).simplify()
)

Eq(uhat(z, mu[k])*vhat(z, mu[k]), (gamma[j] - gamma[k])/(U(z, mu[j])*V(z, mu[j])*a[j]/(U(z, mu[k])*V(z, mu[k])*a[k]) - 1))

It follows that $h$ must be proportional to the weighted difference of modal powers in the new modes. It must also be some weighted sum of all modal powers.

In [97]:
_UV_pow_cons = _uv_pow_cons.subs([_uhat.args, _vhat.args, _uhat.subs(j,k).args, _vhat.subs(j,k).args])
_UV_pow_cons_b = Eq(
    (_UV_pow_cons.lhs/_UV_pow_cons.rhs*h(z)).expand(), 
    _UV_pow_cons.rhs/_UV_pow_cons.rhs*h(z)
)
_UV_pow_cons_b_sum = Eq(
    Sum(U(z, mu[j])*V(z, mu[j])*a[j]/h(z)/4,(j,1,4)),
    U(z, mu[k])*V(z, mu[k])*a[k]/h(z) - gamma[k]
)
_UV_pow_cons_b_sum_h = Eq(
    h(z) ,
    Sum(
        U(z, mu[j])*V(z, mu[j])*a[j]/4*(
            1/gamma[j] - Sum(1/gamma[k]/4,(k,1,4))
        ),
        (j,1,4)
    )
)
_uv_pow_cons_b_sum_h = Eq(
    1,
    Sum(
        uhat(z, mu[j])*vhat(z, mu[j])/4*(
            1/gamma[j] - Sum(1/gamma[k]/4,(k,1,4))
        ),
        (j,1,4)
    )
)

_uv_pow_cons_b_sum_h_check = _uv_pow_cons_b_sum_h.doit().subs([
    (uhat(z, mu[_j+1])*vhat(z, mu[_j+1]), gamma[_j+1] - rho(z)) for _j in range(4)
]).subs(gamma[4], - gamma[1] - gamma[2] - gamma[3]).expand().simplify()
if not _uv_pow_cons_b_sum_h_check:
    raise ValueError('_uv_pow_cons_b_sum_h_check failed')

_UV_pow_cons
_UV_pow_cons_b
_UV_pow_cons_b_sum
_UV_pow_cons_b_sum_h
_uv_pow_cons_b_sum_h

Eq(U(z, mu[j])*V(z, mu[j])*a[j]/h(z) - U(z, mu[k])*V(z, mu[k])*a[k]/h(z), gamma[j] - gamma[k])

Eq(U(z, mu[j])*V(z, mu[j])*a[j]/(gamma[j] - gamma[k]) - U(z, mu[k])*V(z, mu[k])*a[k]/(gamma[j] - gamma[k]), h(z))

Eq(Sum(U(z, mu[j])*V(z, mu[j])*a[j]/(4*h(z)), (j, 1, 4)), U(z, mu[k])*V(z, mu[k])*a[k]/h(z) - gamma[k])

Eq(h(z), Sum((-Sum(1/(4*gamma[k]), (k, 1, 4)) + 1/gamma[j])*U(z, mu[j])*V(z, mu[j])*a[j]/4, (j, 1, 4)))

Eq(1, Sum((-Sum(1/(4*gamma[k]), (k, 1, 4)) + 1/gamma[j])*uhat(z, mu[j])*vhat(z, mu[j])/4, (j, 1, 4)))

In [120]:
Eq(
    Sum(_uvhat.lhs,(j,1,4)), 
    Sum(fraction(_uvhat.rhs)[0],(j,1,4))/fraction(_uvhat.rhs)[1]
)
Eq(
    Sum(_uvhat.lhs/gamma[j],(j,1,4)), 
    Sum(fraction(_uvhat.rhs)[0]/gamma[j],(j,1,4))/fraction(_uvhat.rhs)[1]
)
Eq(
    Sum(gamma[j]*_uvhat.lhs,(j,1,4)), 
    Sum(gamma[j]*fraction(_uvhat.rhs)[0],(j,1,4))/fraction(_uvhat.rhs)[1]
)

Eq(
    -4*rho(z), 
    Sum(fraction(_uvhat.rhs)[0],(j,1,4))/fraction(_uvhat.rhs)[1]
)
Eq(
    4 - rho(z)*Sum(1/gamma[j],(j,1,4)), 
    Sum(fraction(_uvhat.rhs)[0]/gamma[j],(j,1,4))/fraction(_uvhat.rhs)[1]
)
Eq(
    Sum(gamma[j]**2,(j,1,4)), 
    Sum(gamma[j]*fraction(_uvhat.rhs)[0],(j,1,4))/fraction(_uvhat.rhs)[1]
)

Eq(Sum(uhat(z, mu[j])*vhat(z, mu[j]), (j, 1, 4)), Sum(U(z, mu[j])*V(z, mu[j])*a[j], (j, 1, 4))/h(z))

Eq(Sum(uhat(z, mu[j])*vhat(z, mu[j])/gamma[j], (j, 1, 4)), Sum(U(z, mu[j])*V(z, mu[j])*a[j]/gamma[j], (j, 1, 4))/h(z))

Eq(Sum(uhat(z, mu[j])*vhat(z, mu[j])*gamma[j], (j, 1, 4)), Sum(U(z, mu[j])*V(z, mu[j])*a[j]*gamma[j], (j, 1, 4))/h(z))

Eq(-4*rho(z), Sum(U(z, mu[j])*V(z, mu[j])*a[j], (j, 1, 4))/h(z))

Eq(-rho(z)*Sum(1/gamma[j], (j, 1, 4)) + 4, Sum(U(z, mu[j])*V(z, mu[j])*a[j]/gamma[j], (j, 1, 4))/h(z))

Eq(Sum(gamma[j]**2, (j, 1, 4)), Sum(U(z, mu[j])*V(z, mu[j])*a[j]*gamma[j], (j, 1, 4))/h(z))

In [72]:
Eq(
    U(z,mu[j])*V(z, mu[j]),
    solve(
        Eq(
            _UV_pow_cons_b.lhs - _UV_pow_cons_b.lhs.replace(k,l),
            _UV_pow_cons_b.rhs - _UV_pow_cons_b.rhs.replace(k,l)
        ), 
        U(z,mu[j])*V(z, mu[j])
    )[0].expand().collect([U(z,mu[k])*V(z, mu[k]), U(z,mu[l])*V(z, mu[l])], factor)
)

Eq(U(z, mu[j])*V(z, mu[j]), -(gamma[j] - gamma[k])*U(z, mu[l])*V(z, mu[l])*a[l]/((gamma[k] - gamma[l])*a[j]) + (gamma[j] - gamma[l])*U(z, mu[k])*V(z, mu[k])*a[k]/((gamma[k] - gamma[l])*a[j]))

In [104]:
limit(apart((a[0] - a[2])*(a[1] - a[3])/((a[0] - a[3])*(a[1] - a[2])),a[3]).subs([
    (a[0], x),
    (a[1], 1),
    (a[2], 0)
]),a[3],oo)

x

In [110]:
h_uv_sum
UV_to_uv_lam1_hs[0]
UV_to_uv_lam1_hs[1]

UV_normed
UV_uv_normed_sum
UV_uv_normed_sum_h
UV_uv_normed_sum_invert

gamma_jk_uv_h
gamma_jk_uv_h_b
gamma_jk_uv_h_c
gamma_jk_uv_h_e
uvj_h
UV_jk_pow_cons

Eq(h(z), d[5] - Sum((gamma[l] - lambda[1])*U(z, mu[l])*V(z, mu[l]), (l, 1, 4)))

Eq(uhat(z, mu[j]), 2*I*sqrt(-gamma[j] + lambda[1])*U(z, mu[j])*sqrt(lambda[1])/sqrt(h(z)))

Eq(vhat(z, mu[j]), 2*I*sqrt(-gamma[j] + lambda[1])*V(z, mu[j])*sqrt(lambda[1])/sqrt(h(z)))

Eq(uhat(z, mu[j])*vhat(z, mu[j]), -4*(-gamma[j] + lambda[1])*U(z, mu[j])*V(z, mu[j])*lambda[1]/h(z))

Eq(Sum(uhat(z, mu[l])*vhat(z, mu[l]), (l, 1, 4))/4, lambda[1]*Sum((gamma[l] - lambda[1])*U(z, mu[l])*V(z, mu[l]), (l, 1, 4))/h(z))

...

Eq((gamma[j] - gamma[k])*h(z), 4*(gamma[j] - lambda[1])*U(z, mu[j])*V(z, mu[j])*lambda[1] - 4*(gamma[k] - lambda[1])*U(z, mu[k])*V(z, mu[k])*lambda[1])

Eq(Sum(h(z)*gamma[j], (k, 1, 4))/4 + Sum(-h(z)*gamma[k], (k, 1, 4))/4, Sum(4*(gamma[j] - lambda[1])*U(z, mu[j])*V(z, mu[j])*lambda[1], (k, 1, 4))/4 + Sum(-4*(gamma[k] - lambda[1])*U(z, mu[k])*V(z, mu[k])*lambda[1], (k, 1, 4))/4)

Eq(h(z), 4*U(z, mu[j])*V(z, mu[j])*lambda[1] - d[5]*lambda[1]/(gamma[j] - lambda[1]))

Eq(U(z, mu[j])*V(z, mu[j]), h(z)/(4*lambda[1]) + d[5]/(4*(gamma[j] - lambda[1])))

Eq(U(z, mu[j])*V(z, mu[j]) - U(z, mu[k])*V(z, mu[k]), -d[5]/(4*(gamma[k] - lambda[1])) + d[5]/(4*(gamma[j] - lambda[1])))

### Similar as before but not $\lambda_1$

In [137]:
_h_upsilon = Eq(h(z), upsilon[0] + Sum(upsilon[j]*U(z, mu[l])*V(z, mu[l]), (l, 1, 4)))
_uhat = Eq(uhat(z, mu[j]), sqrt(a[j])*U(z, mu[j])/sqrt(h(z)))
_vhat = Eq(vhat(z, mu[j]), sqrt(a[j])*V(z, mu[j])/sqrt(h(z)))
_uvhat = Eq(_uhat.lhs*_vhat.lhs, _uhat.rhs*_vhat.rhs)
_uvhat_cons = Eq(gamma[j] - gamma[k], _uvhat.rhs - _uvhat.rhs.subs(j,k)) 

Eq(
    h(z), 
    solve(Eq(
        x, 
        (h(z)-upsilon[0])/h(z)
    ),h(z))[0].subs(x, Sum(uhat(z, mu[j])*vhat(z, mu[j])*upsilon[j]/a[j],(j,1,4)))
)


_h_upsilon
_uhat
_vhat
_uvhat
_uvhat_cons

Eq(h(z), -upsilon[0]/(Sum(uhat(z, mu[j])*vhat(z, mu[j])*upsilon[j]/a[j], (j, 1, 4)) - 1))

Eq(h(z), upsilon[0] + Sum(U(z, mu[l])*V(z, mu[l])*upsilon[j], (l, 1, 4)))

Eq(uhat(z, mu[j]), U(z, mu[j])*sqrt(a[j])/sqrt(h(z)))

Eq(vhat(z, mu[j]), V(z, mu[j])*sqrt(a[j])/sqrt(h(z)))

Eq(uhat(z, mu[j])*vhat(z, mu[j]), U(z, mu[j])*V(z, mu[j])*a[j]/h(z))

Eq(gamma[j] - gamma[k], U(z, mu[j])*V(z, mu[j])*a[j]/h(z) - U(z, mu[k])*V(z, mu[k])*a[k]/h(z))

In [141]:
__x = Eq(
    _uvhat_cons.lhs*h(z)*upsilon[j]/a[j], 
    (_uvhat_cons.rhs*h(z)*upsilon[j]/a[j]).expand()
)

__x

Eq((gamma[j] - gamma[k])*h(z)*upsilon[j]/a[j], U(z, mu[j])*V(z, mu[j])*upsilon[j] - U(z, mu[k])*V(z, mu[k])*a[k]*upsilon[j]/a[j])

In [150]:
Eq(
    (Sum(gamma[j]*upsilon[j]/a[j],(j,1,4))
     - gamma[k]*Sum(upsilon[j]/a[j],(j,1,4)) -1)*h(z) 
     + upsilon[0],
     - U(z, mu[k])*V(z, mu[k])*a[k]*Sum(upsilon[j]/a[j],(j,1,4))
)

__xx1 = Eq(
    h(z) 
     ,
     (- upsilon[0] - U(z, mu[k])*V(z, mu[k])*a[k]*Sum(upsilon[j]/a[j],(j,1,4)))/
    (Sum(gamma[j]*upsilon[j]/a[j],(j,1,4)) - gamma[k]*Sum(upsilon[j]/a[j],(j,1,4)) -1)
)

__xx1
Eq(
    0 
     ,
     __xx1.rhs - __xx1.rhs.subs(k,l)
)

Eq(
    4*(Sum(gamma[j]*upsilon[j]/a[j],(j,1,4)) -1)*h(z) 
     + 4*upsilon[0],
     - Sum(U(z, mu[k])*V(z, mu[k])*a[k],(k,1,4))*Sum(upsilon[j]/a[j],(j,1,4))
)

Eq((-gamma[k]*Sum(upsilon[j]/a[j], (j, 1, 4)) + Sum(gamma[j]*upsilon[j]/a[j], (j, 1, 4)) - 1)*h(z) + upsilon[0], -U(z, mu[k])*V(z, mu[k])*a[k]*Sum(upsilon[j]/a[j], (j, 1, 4)))

Eq(h(z), (-U(z, mu[k])*V(z, mu[k])*a[k]*Sum(upsilon[j]/a[j], (j, 1, 4)) - upsilon[0])/(-gamma[k]*Sum(upsilon[j]/a[j], (j, 1, 4)) + Sum(gamma[j]*upsilon[j]/a[j], (j, 1, 4)) - 1))

Eq(0, (-U(z, mu[k])*V(z, mu[k])*a[k]*Sum(upsilon[j]/a[j], (j, 1, 4)) - upsilon[0])/(-gamma[k]*Sum(upsilon[j]/a[j], (j, 1, 4)) + Sum(gamma[j]*upsilon[j]/a[j], (j, 1, 4)) - 1) - (-U(z, mu[l])*V(z, mu[l])*a[l]*Sum(upsilon[j]/a[j], (j, 1, 4)) - upsilon[0])/(-gamma[l]*Sum(upsilon[j]/a[j], (j, 1, 4)) + Sum(gamma[j]*upsilon[j]/a[j], (j, 1, 4)) - 1))

Eq((4*Sum(gamma[j]*upsilon[j]/a[j], (j, 1, 4)) - 4)*h(z) + 4*upsilon[0], -Sum(upsilon[j]/a[j], (j, 1, 4))*Sum(U(z, mu[k])*V(z, mu[k])*a[k], (k, 1, 4)))

In [152]:
Cxz = Eq(C, (a[0]*x+a[1])/(a[2]*x + a[3]) - (b[0]*z+b[1])/(b[2]*z + b[3]))

Cxz

Eq(C, (x*a[0] + a[1])/(x*a[2] + a[3]) - (z*b[0] + b[1])/(z*b[2] + b[3]))

In [155]:
_den = fraction(Cxz.simplify().rhs)[1]

_den

(x*a[2] + a[3])*(z*b[2] + b[3])

In [159]:
Eq((Cxz.lhs*_den - Cxz.rhs*_den).expand().simplify().collect([x*z,x,z], factor),0)

Eq(C*a[3]*b[3] + x*z*(C*a[2]*b[2] - a[0]*b[2] + a[2]*b[0]) + x*(C*a[2]*b[3] - a[0]*b[3] + a[2]*b[1]) + z*(C*a[3]*b[2] - a[1]*b[2] + a[3]*b[0]) - a[1]*b[3] + a[3]*b[1], 0)

In [164]:
C_lim_xz_inf = Eq(C,limit(apart(limit(apart(Cxz.rhs,x),x,oo),z),z,oo))
C_lim_xz_inf

Eq(C, (a[0]*b[2] - a[2]*b[0])/(a[2]*b[2]))

In [167]:
Cxz_2 = Cxz.subs(a[0],solve(C_lim_xz_inf,a[0])[0])
Cxz_2

Eq(C, -(z*b[0] + b[1])/(z*b[2] + b[3]) + (x*(C*b[2] + b[0])*a[2]/b[2] + a[1])/(x*a[2] + a[3]))